In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib_venn import venn2, venn2_circles
import argparse
from create_reference_from_tsv_and_pepxml import ReferenceWriter
from sensitivity_calculator import SensitivityAndSpecificity
from create_PSM_df import PSM_FDR
from Bio import SeqIO
from collections import defaultdict
from Bio import pairwise2 as pw2
import wget

In [2]:
# load taxon graph
import sys  
sys.path.insert(0, '/home/jules/tax2proteome_projects/tax2proteome/')
from TaxonGraph import TaxonGraph
taxon_graph = TaxonGraph()
taxon_graph.create_graph("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/taxdump.tar.gz")

In [3]:
# Files
#new_reduced: ohne +1 charge specta
uniprot_nr_species_tsv_with_wrong_error = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/uniprot/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_new_reduced.tsv"


reference_tsv_with_kleiner_db = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_db/Run1_U1_2000ng.t.xml_reduced.tsv"
reference_tsv_with_aradiopsis = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_reference_aradiopsis/Run1_U1_2000ng_kleiner_aradiopsis.t.xml_reduced.tsv"
reference_tsv_with_ara_yeast_human = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_ara_yeast_human/Run1_U1_2000ng.t.xml_new_reduced.tsv"
path_to_bachelor_results = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/"
path_to_uniprot_reanalysis = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_reanalysis_uniprot"
path_to_tanca_results_bachelor = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv"
path_to_kleiner_species_nr_0_9_identity = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/uniprot_species_sequences_0_9_identity/Run1_U1_2000ng_uniprot_species_0_9_identity.t.xml.tsv"
path_to_kleiner_species_nr_0_9_identity_reduced = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/uniprot_species_sequences_0_9_identity/Run1_U1_2000ng_uniprot_species_0_9_identity.t.xml_new_reduced.tsv"
def get_path_to_refernce_analysis_results(fdr):
    return f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_db/Run1_U1_2000ng.t.xml_reduced.tsv_{fdr}_sensitivity_new"

def get_path_to_refernce_analysis_results_with_aradiopsis(fdr):
    return f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_kleiner_reference_aradiopsis/Run1_U1_2000ng_kleiner_aradiopsis.t.xml_reduced.tsv_{fdr}_sensitivity_new"



uniprot_reanalysis = {'species': path_to_uniprot_reanalysis + "/species/Run1_U1_2000ng_uniprot_species.t.xml_new_reduced.tsv",
                      'species_nr': path_to_uniprot_reanalysis + "/species_nr/Run1_U1_2000ng_uniprot_species_nr.t.xml_reduced.tsv",
                      "species_species": path_to_uniprot_reanalysis + "/species_species/Run1_U1_2000ng_uniprot_species_species.t.xml_reduced.tsv",
                      'genus_nr': path_to_uniprot_reanalysis + "/genus_nr/Run1_U1_2000ng_uniprot_genus_nr.t.xml_reduced.tsv"
                     }
    
uniprot_nr_reduced_tsv = {
    'subspecies': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus_nr.t.xml_new_reduced.tsv",
    'family': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_family_nr.t.xml_new_reduced.tsv",
}
    
uniprot_reduced_tsv = {
    'subspecies': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus.t.xml_new_reduced.tsv"
}

uniprot_species_reduced_tsv =  {
    'subspecies': path_to_uniprot_reanalysis + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_species.t.xml_new_reduced.tsv",
    'genus':  path_to_bachelor_results + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus_species.t.xml_new_reduced.tsv"
}

ncbi_reduced_tsv= {
    'subspecies': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_genus.t.xml_new_reduced.tsv"
}

ncbi_species_reduced_tsv = {
    'species': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_species_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/ncbi_kleiner/x_tandem_tsv/Run1_U1_2000ng_ncbi_kleiner_genus_species.t.xml_new_reduced.tsv"
}
swissprot_reduced_tsv = {
    'species': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_species.t.xml_new_reduced.tsv",
    'genus': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_genus.t.xml_new_reduced.tsv",
    'family': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_family.t.xml_new_reduced.tsv",
    'order': path_to_bachelor_results + "/swissprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_swissprot_order.t.xml_new_reduced.tsv"
}

tanca_ncbi_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_ncbi_Tanca_species.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_ncbi_tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_ncbi_tanca_family.t.xml_new_reduced.tsv",
}

tanca_uniprot_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_species_nr.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_family_nr.t.xml_new_reduced.tsv",
    'order': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_order_nr.t.xml_new_reduced.tsv"
}

tanca_swissprot_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_species.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_family.t.xml_new_reduced.tsv",
    'order': path_to_tanca_results_bachelor + "/9MM_FASP_swissprot_tanca_order.t.xml_new_reduced.tsv"
}

def get_dict_databases_to_size_and_result_file(fdr):
    dict_databases_to_size_and_result_file={'reference': (123088, f"{reference_tsv_with_kleiner_db}_{fdr}_sensitivity_new"),
                                        "reference_with_aradiopsis": (138980, f"{reference_tsv_with_aradiopsis}_{fdr}_sensitivity_new"),
                                        
                                        'ncbi_subspecies': (300000, f"{ncbi_reduced_tsv['subspecies']}_{fdr}_sensitivity_new"),
                                        'ncbi_species': (8702135, f"{ncbi_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "ncbi_genus":(27804893, f"{ncbi_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        
                                        "uniprot_subspecies": (4683371, f"{uniprot_reduced_tsv['subspecies']}_{fdr}_sensitivity_new"),
                                        "uniprot_species": (4683371, f"{uniprot_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "uniprot_species_species": (2093157, f"{uniprot_species_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "uniprot_subspecies_nr": (200000, f"{uniprot_nr_reduced_tsv['subspecies']}_{fdr}_sensitivity_new"),
                                        "uniprot_species_nr": (2991727, f"{uniprot_nr_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "uniprot_genus": (18352148,  f"{uniprot_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        "uniprot_genus_species": (13068285,  f"{uniprot_species_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        "uniprot_genus_nr": (13210287,  f"{uniprot_nr_reduced_tsv['genus']}_{fdr}_sensitivity_new"),
                                        "uniprot_family_nr": (22509624, f"{uniprot_nr_reduced_tsv['family']}_{fdr}_sensitivity_new"),
                                        
                                        "swissprot_species": (58505, f"{swissprot_reduced_tsv['species']}_{fdr}_sensitivity_new"),
                                        "swissprot_genus": (88164, f"{swissprot_reduced_tsv['genus']}_{fdr}_sensitivity_new" ),
                                        "swissprot_family": (124044, f"{swissprot_reduced_tsv['family']}_{fdr}_sensitivity_new"),
                                        "swissprot_order": (181725, f"{swissprot_reduced_tsv['order']}_{fdr}_sensitivity_new")
                                       }
    return dict_databases_to_size_and_result_file

dict_databases_to_size_and_result_file = get_dict_databases_to_size_and_result_file(0.05)
uniprot_dict =dict(filter(lambda item: item[0] in ['uniprot_subspecies', 'uniprot_species', 'uniprot_genus'], dict_databases_to_size_and_result_file.items()))
uniprot_nr_dict =dict(filter(lambda item: '_nr' in item[0], dict_databases_to_size_and_result_file.items()))
uniprot_species_dict=dict(filter(lambda item: item[0] in ['uniprot_species_species', 'uniprot_genus_species'], dict_databases_to_size_and_result_file.items()))
ncbi_dict =dict(filter(lambda item: 'ncbi' in item[0], dict_databases_to_size_and_result_file.items()))
swissprot_dict=dict(filter(lambda item: 'swiss' in item[0], dict_databases_to_size_and_result_file.items()))

In [4]:
def get_all_rows_without_charge1(title_col):
    return [title.split('.')[-1] != '' for title in title_col]

def remove_spectra_of_charge_one_from_reduced_tsv(reduced_tsv):
    return reduced_tsv[get_all_rows_without_charge1(reduced_tsv.Title)]
def get_decoy_rows_from_decoy_column(decoy_column):
    return [True if d=={True} else False for d in decoy_column]

def get_tax_rows(tax_column, taxid):
    return [True if tax_set == {taxid} else False for tax_set in tax_column]

def get_decoy_rows_from_protein_column(protein_column):
    return [True if 'REVERSED' in p else False for p in protein_column]


def get_hit_rows(protein_column):
    return [False if 'REVERSED' in p else True for p in protein_column]

def get_decoy_rows2(decoy_column):
    return [True if d_set in [{True, False}, {True}] else False for d_set in decoy_column]

def get_hit_rows2(decoy_column):
    return [True if d_set in  [{True, False}, {False}] else False for d_set in decoy_column]

def get_not_crap_rows(column_name):
    return [True if t_set != {"CRAP"} else False for t_set in column_name]

def get_taxa_rows(column_name, taxID):
    if type(taxID)==int:
        return [True if taxID in t_set else False for t_set in column_name]
    elif type(taxID)==list:
        return [True if len(set(taxID).intersection(t_set))>0 else False for t_set in column_name]

def get_spectra_rows(spectra_column, spectra_set):
    return [True if spectrum in spectra_set else False for spectrum in spectra_column]
def get_not_spectra_rows(spectra_column, spectra_set):
    return [True if spectrum not in spectra_set else False for spectrum in spectra_column]

def get_single_acc_rows(acc_column, acc):
    return [True if acc in acc_set else False for acc_set in acc_column]
            
def get_acc_rows(acc_column, acc_set1, all_acc_contained=True):
    """
    returns rows, where ALL acc in row also in acc_set1
    """
    if all_acc_contained:
        return [True if len(acc_set2.difference(acc_set1)) == 0 else False for acc_set2 in acc_column]
    else:
        return [True if len(acc_set2.intersection(acc_set1)) > 0 else False for acc_set2 in acc_column]

def get_not_acc_rows(acc_column, acc_set1, all_acc_contained=True):
    if all_acc_contained:
        return [False if len(acc_set2.difference(acc_set1)) == 0  else True for acc_set2 in acc_column]
    else:
        return [False if len(acc_set2.intersection(acc_set1)) > 0  else True for acc_set2 in acc_column]

In [5]:
def read_result_df(f):
    df =  ReferenceWriter.read_csv_with_generic_function(f,['Hyperscore', 'taxID', 'decoy'])
    return df[['Title', 'Peptide', 'Hyperscore', 'taxID', 'decoy']]
    
def get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra=False, columns=None):
    cs = ['Protein', 'Hyperscore', 'decoy', 'taxID']
    if columns:
        cs = cs + columns
    reduced_df = ReferenceWriter.read_csv_with_generic_function(file, cs, remove_one_charged_spectra)
    fdr_pos_result, number_psm_result, number_decoy_result, double_spectra_result, score_last_item_result = PSM_FDR.determine_FDR_position(reduced_df, fdr)
    return number_psm_result, reduced_df[0:fdr_pos_result]

def get_df_in_fdr_without_decoy(file, fdr, remove_one_charged_spectra=True, columns=None):
    df = get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra, columns)[1]
    df = df[get_hit_rows2(df.decoy)]
    return df

In [6]:
def explode_df_and_remove_crap_and_decoy(df, column_to_explode):
    df_without_set = df.explode(column_to_explode).reset_index(drop=True)
    df_without_crap = df_without_set[[True if t_set != "CRAP" else False for t_set in df_without_set[column_to_explode]]]
    df_without_decoy = df_without_crap[[True if t_set != "DECOY" else False for t_set in df_without_crap[column_to_explode]]]
    return df_without_decoy

In [7]:
def count_taxa(df_column):
    taxa_count_dict = {}
    for tax_set in df_column:
        for taxon in tax_set:
            if taxon in taxa_count_dict:
                taxa_count_dict[taxon]+= 1
            else:
                taxa_count_dict[taxon]=1
    return taxa_count_dict

def get_taxon_specific_accs(tax, db):
    taxon_accs = set()
    len_db=len(db)
    all_descending_taxa = taxon_graph.find_taxIDs(tax)
    search_strs = [f'OX={taxon}' for taxon in all_descending_taxa]
    print(len(all_descending_taxa))
    for i, entry in enumerate(db):
        if (len_db/(i+1))%10==0:
            print(f"{(i+1)/len_db *100} % db searched")
        if any(taxon in entry.description for taxon in search_strs):
            taxon_accs.add(entry.id)
    return taxon_accs

def get_taxon_specific_accs2(tax, df_acc_taxon):
    taxon_accs = set()
    all_descending_taxa = taxon_graph.find_taxIDs(tax)
    print(len(all_descending_taxa))
    for taxon in all_descending_taxa:
        taxon_accs.add(entry.id)
    return taxon_accs

In [8]:
def compare_seq(acc1, acc2):
    seq1 = wget(f"https://www.uniprot.org/uniref/{acc1}.fasta")
    seq2 = wget(f"https://www.uniprot.org/uniref/{acc2}.fasta")
    global_align = pw2.align.globalxx(first_seq, second_seq)
    number_matching_AS = global_align[0][2]
    percent_matcing = number_matching_seq/len(seq1)*100
    return percent_matching

In [299]:
def get_accs(tax):
    all_descending_taxa = taxon_graph.find_taxIDs(tax)
    #print(sorted(list(all_descending_taxa)))
    acc_set = set()
    with open("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/acc2tax_uniprot", 'r') as infile:
        for line in infile:
            fields = line.split('\t')
            try: 
                taxon = int(fields[1].strip())
                if taxon in all_descending_taxa:
                    acc_set.add(fields[0].strip()) 
            # empty lines
            except:
                continue
    return acc_set

def get_taxa_specific_accs(taxids):
    if type(taxids)==int:
        taxa_specific_accs = get_accs(taxids)
    elif type(taxids)==list:
        taxa_specific_accs = set()
        for taxon in taxids:
            taxa_specific_accs = taxa_specific_accs.union(get_accs(taxon))
    return taxa_specific_accs

In [288]:
def remove_acc_row(column, acc_to_remove_set):
    return [False if len(acc_set.difference(acc_to_remove_set))==0 else True for acc_set in column]            
   
def remove_all_once_appearing_accs(df):
    #print(len(df))
    all_accs = [item for sublist in df.Protein for item in sublist]
    all_accs.sort()
    #print("all_accs: ", len(set(all_accs)))
    acc_before=""
    acc_to_keep=set()
    for acc in all_accs:
        if acc == acc_before:
            acc_to_keep.add(acc)
        acc_before = acc
    print("acc_to_keep ", len(acc_to_keep))
    accs_to_remove = set(all_accs).difference(acc_to_keep)
    print("accs_to_remove: ", len(accs_to_remove))
    df=df[remove_acc_row(df.Protein, accs_to_remove)]
    df.Protein = df.Protein.apply(lambda acc_set: acc_set.intersection(acc_to_keep))
    return df


In [250]:
accs_to_remove= {1, 2, 3, 4}
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data={'Protein':[{11,12,10},{6},{1},{1,2, 3, 5}, {2,3, 8, 9}]})
print(df)
df = remove_all_once_appearing_accs(df)
print(df)
#print(df)
#df = remove_all_once_appearing_taxa(df, "Protein")
#print(df)
#all_accs:  1618
#accs_to_remove: 1386
#12

        Protein
0  {10, 11, 12}
1           {6}
2           {1}
3  {1, 2, 3, 5}
4  {8, 9, 2, 3}
acc_to_keep  3
accs_to_remove:  7
     Protein
2        {1}
3  {1, 2, 3}
4     {2, 3}


/home/jules/.local/lib/python3.9/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
# wie oft neu identifizierte in allen spektren
def get_spectra_to_accs(df_family, new_identified_accs, df_species):
    spectra_set =set()
    for row in df_family.values.tolist():
        if any(acc in row[1] for acc in new_identified_accs):
            spectra_set.add(row[0])
    return len(df_species[df_species.Title.isin(spectra_set)])
    

def get_new_spectra_characteristics(family_taxon, df_family_taxon_not_in_species_df, df_family, df_species):
    # Protein muss als set geladen sein
    taxa_specific_accs = get_accs(family_taxon)
    new_identified_accs = [acc.split('|')[1] for acc_set in list(df_family_taxon_not_in_species_df.Protein) \
                           for acc in acc_set]
    #new_identified_accs = [acc.split('|')[1] for acc in new_identified_accs]
    new_identified_accs_fam_tax = taxa_specific_accs.intersection(set(new_identified_accs))
    print(f"Number of new identified accs on family level for taxon{family_taxon}: {len(new_identified_accs_fam_tax)}")
    
    acc_to_count_dict = {}
    for acc in new_identified_accs_fam_tax:
        count = sum([acc in acc_long for acc_long_set in df_family_taxon_not_in_species_df.Protein \
                     for acc_long in acc_long_set])
        acc_to_count_dict[acc]=count
    
    acc_count_0_dict={}
    acc_count_1_dict={}
    acc_count_2_dict={}
    acc_count_3_dict={}
    acc_count_4_dict={}
    acc_count_5_dict={}
    for acc, count in acc_to_count_dict.items():
        if count ==1 :
            acc_count_1_dict[acc]=count
        elif count==2:
            acc_count_2_dict[acc]=count
        elif count==3:
            acc_count_3_dict[acc]=count
        elif count==4:
            acc_count_4_dict[acc]=count
        elif count>4:
            acc_count_5_dict[acc]=count
        elif count ==0:
            acc_count_0_dict[acc]=count
            
            
    print(f"Anzahl Protein-Identifikationen auf Family Level für die mit taxon {family_taxon} neu hinzugekommen accs\n\
    mit 1 identifikation:{len(acc_count_1_dict)}, }\nmit 2 identifikationen: {len(acc_count_2_dict)},\n\
    mit 3 identifikationen: {len(acc_count_3_dict)}, \nmit 4 identifikationen: {len(acc_count_4_dict)}, \n\
    mit mehr als 4 identifikationen: {len(acc_count_5_dict)}") 
    # davon schon im species df identifiziere spektren
    df = df_family[['Title','Protein']]
    df = df.astype({'Protein': 'string'})

    print(f"Anzahl der identifizierten Spektren, welche bereits auf species Level durch Proteine identifiziert wurden, welche auch in den neuen Identifikationen enthalten sind.")
    print(f"Proteine die auf species Level 0 mal identifiziert wurden: {get_spectra_to_accs(df, acc_count_0_dict.keys(), df_species)}.")
    print(f"Proteine die auf species Level 1 mal identifiziert wurden: {get_spectra_to_accs(df, acc_count_1_dict.keys(), df_species)}.")
    print(f"Proteine die auf species Level 2 mal identifiziert wurden:: {get_spectra_to_accs(df, acc_count_2_dict.keys(), df_species)}.")
    print(f"Proteine die auf species Level 3 mal identifiziert wurden:: {get_spectra_to_accs(df, acc_count_3_dict.keys(), df_species)}.")    
    print(f"Proteine die auf species Level 4 mal identifiziert wurden:: {get_spectra_to_accs(df, acc_count_4_dict.keys(), df_species)}.")
    print(f"Proteine die auf species Level mehr als 4 mal identifiziert wurden:: {get_spectra_to_accs(df, acc_count_5_dict.keys(), df_species)}.")            

In [83]:
def get_species_spectra_characteristics(taxid_species, df_in_fdr_uniprot_species):
    # Protein muss als set geladen sein
    if type(taxid_species)==int:
        taxa_specific_accs = get_accs(taxid_species)
    elif type(taxid_species)==list:
        taxa_specific_accs = set()
        for taxon in taxid_species:
            taxa_specific_accs = taxa_specific_accs.union(get_accs(taxon))
    df_taxa = df_in_fdr_uniprot_species[get_taxa_rows(df_in_fdr_uniprot_species.taxID_species, taxid_species)]
    taxa_identified_accs = [acc.split('|')[1] for acc_set in list(df_taxa.Protein) \
                               for acc in acc_set]
    #new_identified_accs = [acc.split('|')[1] for acc in new_identified_accs]
    identified_accs = taxa_specific_accs.intersection(set(taxa_identified_accs))
    print(f"Number of identified accs on species level for taxon{taxid_species}: {len(identified_accs)}")
    acc_to_count_dict = {}
    for acc in identified_accs:
        count = sum([acc in acc_long for acc_long_set in df_taxa.Protein for acc_long in acc_long_set])
        acc_to_count_dict[acc]=count

    acc_count_0_dict={}
    acc_count_1_dict={}
    acc_count_2_dict={}
    acc_count_3_dict={}
    acc_count_4_dict={}
    for acc, count in acc_to_count_dict.items():
        if count ==1 :
            acc_count_1_dict[acc]=count
        elif count==2:
            acc_count_2_dict[acc]=count
        elif count==3:
            acc_count_3_dict[acc]=count
        elif count>3:
            acc_count_4_dict[acc]=count
        elif count ==0:
            acc_count_0_dict[acc]=count


    print(f"Anzahl Protein-Identifikationen auf Species Level für taxon {taxid_species} accs \n\
    mit 1 identifikation:{len(acc_count_1_dict)}, \nmit 2 identifikationen: {len(acc_count_2_dict)},\n\
    mit 3 identifikationen: {len(acc_count_3_dict)}, \nmit mehr als 3 identifikationen: {len(acc_count_4_dict)}") 
    print('spectra characteristica for exclusive spectra: ')
    if type(taxid_species)==int:
        df = df_taxa[df_taxa.taxID_species == {taxid_species}]
    elif type(taxid_species)==list:
        df = df_taxa[(df_taxa.taxID_species == set(taxid_species)) | (df_taxa.taxID_species == {taxid_species[0]}) | (df_taxa.taxID_species == {taxid_species[1]})]
    exclusive_identified_accs = [acc.split('|')[1] for acc_set in list(df.Protein) for acc in acc_set]

    #new_identified_accs = [acc.split('|')[1] for acc in new_identified_accs]
    exclusive_identified_accs = taxa_specific_accs.intersection(set(exclusive_identified_accs))
    print(f"Number of identified accs on species level for exclusive taxon {taxid_species}: {len(exclusive_identified_accs)}")  

In [11]:
def describe_difference_between_species_and_family_identification(name, list_taxids_species, taxid_family, df_in_fdr_species, df_in_fdr_family):
    print(name)
    df_species_tax_spec = df_in_fdr_species[get_taxa_rows(df_in_fdr_species.taxID_species, list_taxids_species)]
    df_family_tax_fam = df_in_fdr_family[get_taxa_rows(df_in_fdr_family.taxID_family, taxid_family)]
    print(f"identified spectra species for taxon {list_taxids_species}: {len(set(df_species_tax_spec.Title))} \
          \n identified spectra family for taxon {taxid_family}: {len(set(df_family_tax_fam.Title))}")
    df_family_tax_fam_not_in_species_df=df_family_tax_fam[~df_family_tax_fam.Title.isin(df_species_tax_spec.Title)]
    print(f"all new identified spectra family  {len(set(df_family_tax_fam_not_in_species_df.Title))}")
    df_only_one_taxa_tax_fam = df_family_tax_fam_not_in_species_df[df_family_tax_fam_not_in_species_df.taxID_family == {taxid_family}]
    taxa_count_dict_tax_fam = count_taxa(df_family_tax_fam_not_in_species_df.taxID_family)
    print(f"identified exclusive for taxon {taxid_family}: {len(set(df_only_one_taxa_tax_fam.Title))}\n \
    not exclusive identified for taxon {taxid_family} taxid to PSM number dict: {taxa_count_dict_tax_fam}")
    df_family_tax_fam_in_species_df = df_family_tax_fam_not_in_species_df[df_family_tax_fam_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
    print(f"Davon wurde bereits vorher für andere spezien identifiziert: {len(set(df_family_tax_fam_in_species_df.Title))} spektren.")
    return df_family_tax_fam_not_in_species_df, df_only_one_taxa_tax_fam

In [261]:
def count_accs(acc_list):
    acc_list.sort
    count_dict={1:0, 2:0, 3:0, '>3':0}
    acc_before = acc_list[0]
    count=0
    for acc in acc_list:
        if acc == acc_before:
            count+=1
        else:
            if count > 3:
                count_dict['>3'] = count_dict['>3']+1
            else:
                count_dict[count] = count_dict[count]+1
            count=1    
        acc_before = acc
    return count_dict

def describe_difference_between_species_and_family_identification_with_removed_accs(name, taxids_species, taxid_family, df_in_fdr_species, df_in_fdr_family):
    print(name)
    #load accs of taxon {'A0A3A5I3L3', 'A0A4T2HSI9', 'A0A4R6HX40', 'G9LQ63', 'A0A410ZMS7' ...}
    species_taxa_specific_accs = get_taxa_specific_accs(taxids_species)
    family_taxa_specific_accs = get_taxa_specific_accs(taxid_family)
    # keep only specific accs
    df_in_fdr_species_tax_acc = df_in_fdr_uniprot_species.copy(deep=True)
    df_in_fdr_family_tax_acc = df_in_fdr_uniprot_family.copy(deep=True)
    df_in_fdr_species_tax_acc.Protein = df_in_fdr_species_tax_acc.Protein.apply(lambda acc_set: \
                                                            acc_set.intersection(species_taxa_specific_accs))
    df_in_fdr_family_tax_acc.Protein = df_in_fdr_family_tax_acc.Protein.apply(lambda acc_set: \
                                                            acc_set.intersection(family_taxa_specific_accs))
    #remove rows with no accs
    df_in_fdr_species_tax_acc = df_in_fdr_species_tax_acc[df_in_fdr_species_tax_acc.Protein != set()]
    df_in_fdr_family_tax_acc = df_in_fdr_family_tax_acc[df_in_fdr_family_tax_acc.Protein != set()]
    #print(df_in_fdr_family_186817_acc.head(10))
    # all accs
    accs_spec = sorted([item for sublist in df_in_fdr_species_tax_acc.Protein for item in sublist])
    accs_fam = sorted([item for sublist in df_in_fdr_family_tax_acc.Protein for item in sublist])
    
    print(f"identified spectra species for taxon {taxids_species}: {len(set(df_in_fdr_species_tax_acc.Title))}\n\
    identified spectra family for taxon {taxid_family}: {len(set(df_in_fdr_family_tax_acc.Title))}")
    print(f"identified accs species for taxon {taxids_species}: {len(set(accs_spec))}\n\
    identified accs family for taxon {taxid_family}: {len(set(accs_fam))}")
    
    #count    
    count_dict_spec = count_accs(accs_spec)
    print(f"Anzahl Protein-Identifikationen auf Species Level für taxon {taxids_species}\n\
    mit 1 identifikation:{count_dict_spec[1]}\n\
    mit 2 identifikationen: {count_dict_spec[2]},\n\
    mit 3 identifikationen: {count_dict_spec[3]}, \n\
    mit mehr als 3 Identifikationen: {count_dict_spec['>3']}")     
    
    count_dict_fam = count_accs(accs_fam)
    print(f"Anzahl Protein-Identifikationen auf Family Level für taxon {taxid_family}\n\
    mit 1 identifikation:{count_dict_fam[1]}\n\
    mit 2 identifikationen: {count_dict_fam[2]},\n\
    mit 3 identifikationen: {count_dict_fam[3]}, \n\
    mit mehr als 3 Identifikationen: {count_dict_fam['>3']}") 

    print()
    print("Eigenschaften der neu identifizierten Spektra:")
    df_family_tax_fam_not_in_species_df=df_in_fdr_family_tax_acc[~df_in_fdr_family_tax_acc.Title.isin(df_in_fdr_species_tax_acc.Title)]
    print(f"all new identified spectra family: {len(set(df_family_tax_fam_not_in_species_df.Title))}")
    df_only_one_taxa_tax_fam = df_family_tax_fam_not_in_species_df[df_family_tax_fam_not_in_species_df.taxID_family == {taxid_family}]
    df_only_one_taxa_tax_fam = df_family_tax_fam_not_in_species_df[df_family_tax_fam_not_in_species_df.taxID_family == {taxid_family}]
    taxa_count_dict_tax_fam = count_taxa(df_family_tax_fam_not_in_species_df.taxID_family)
    print(f"identified exclusive for taxon {taxid_family}: {len(set(df_only_one_taxa_tax_fam.Title))}\n\
    not exclusive identified for taxon {taxid_family}, taxid to PSM number dict: \n\
    {taxa_count_dict_tax_fam}")
    df_family_tax_fam_in_species_df = df_family_tax_fam_not_in_species_df[df_family_tax_fam_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
    print(f"Anzal Spekteb bereits vorher für andere Spezien identifiziert: {len(set(df_family_tax_fam_in_species_df.Title))}")


In [12]:
def count_psm_accs(df, set_accs):
    result_dict={}
    result_dict_2={}
    for acc in set_accs:
        df_acc = df[get_single_acc_rows(df.Protein, acc)]
        result_dict[acc]=len(df_acc)
    for count_result in set(result_dict.values()):
        temp = {k:v for (k,v) in result_dict.items() if v==count_result}
        result_dict_2[count_result] = len(temp.keys())
    temp = {k:v for (k,v) in result_dict.items() if v>4}
    result_dict_2['>4'] = len(temp)
    return result_dict_2

In [36]:
#Mehr als 2 PSM pro Protein, wie viele davon in Spezien Identifikation
def get_accs_with_more_than_2_psms(taxid_family, taxid_species, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species):
    df_all_family_accs = df_in_fdr_uniprot_family[get_taxa_rows(df_in_fdr_uniprot_family.taxID_family, taxid_family)]
    # hier bekommt man auch die accs die nicht zur family gehören
    set_all_family_accs = {acc for acc_set in df_all_family_accs.Protein for acc in acc_set}
    result_dict={}
    for acc in set_all_family_accs:
        df_acc = df_in_fdr_uniprot_family[get_single_acc_rows(df_in_fdr_uniprot_family.Protein, acc)]
        result_dict[acc]=len(df_acc)
    temp = {k:v for (k,v) in result_dict.items() if v>2}
    acc_more_than_2_family = set(temp.keys())

    df_all_species_accs = df_in_fdr_uniprot_species[get_taxa_rows(df_in_fdr_uniprot_species.taxID_species, taxid_species)]
    set_all_species_accs = {acc for acc_set in df_all_species_accs_1280.Protein for acc in acc_set}
    result_dict_2={}
    for acc in set_all_species_accs:
        df_acc = df_in_fdr_uniprot_species[get_single_acc_rows(df_in_fdr_uniprot_species.Protein, acc)]
        result_dict[acc]=len(df_acc)
    temp_2 = {k:v for (k,v) in result_dict.items() if v>2}
    acc_more_than_2_species = set(temp_2.keys())
    print(f"Taxid_family: {taxid_family}, Accs auf family level mit mehr als 2 PSMs: {len(acc_more_than_2_family)},\n\
    Taxid_species: {taxid_species},Accs auf species level mit mehr als 2 PSMs: {len(acc_more_than_2_species)},\n\
    intersection: {len(acc_more_than_2_family.intersection(acc_more_than_2_species))}")
    return set_all_family_accs, set_all_species_accs, acc_more_than_2_family, acc_more_than_2_species

In [42]:
def exclusive_psms_of_family_level_in_species(df_exclusive_new_psms, df_in_fdr_uniprot_species, taxid_species):
    set_all_family_psms_exclusive = set(df_exclusive_new_psms.Title)
    set_in_species = set(df_in_fdr_uniprot_species[get_taxa_rows(df_in_fdr_uniprot_species.taxID_species, taxid_species)].Title)
    print(f"Anzahl exclusiver {taxid_species} PSMs family level: {len(set_all_family_psms_exclusive)}, \
    davon schon in species df für 1280 identifiziert: {len(set_all_family_psms_exclusive.intersection(set_in_species))}")

In [13]:
########## KLEINER #########

In [32]:
fdr=0.05
df_in_fdr_ref = get_df_in_fdr_without_decoy(reference_tsv_with_kleiner_db, fdr, columns=['taxID_species', 'Protein']) 
print(len(set(df_in_fdr_ref.Title)))
print(len(set(df_in_fdr_ref[df_in_fdr_ref.decoy != {True}].Title)))

In [13]:
file=uniprot_nr_reduced_tsv['species']
fdr=0.05
df_uni_species = get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra=True, columns=["taxID_species"])[1]

In [176]:
# Vergleich unterschiedlich identfizierte Bakterien species vs. family
# species_df mean score: 41.4, median: 38.5; family_df mean score: 44.1, median: 41.1
fdr=0.05
df_in_fdr_uniprot_species = get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['species'], fdr, columns=['taxID_species']) 
df_in_fdr_uniprot_family =  get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['family'], fdr, columns=['taxID_family'])
print(df_in_fdr_uniprot_family.mean())
print(df_in_fdr_uniprot_family.median())
print(len(set(df_in_fdr_uniprot_species.Title)))
print(len(set(df_in_fdr_uniprot_species[df_in_fdr_uniprot_species.decoy != {True}].Title)))
cs = ['Protein', 'Hyperscore', 'decoy', 'taxID', 'taxID_species']
df_full_uniprot_species = ReferenceWriter.read_csv_with_generic_function(uniprot_nr_reduced_tsv['species'], cs, remove_one_charged_spectra=True)

Unnamed: 0    20934.827974
Hyperscore       44.108415
dtype: float64
Unnamed: 0    20736.5
Hyperscore       41.1
dtype: float64
39193
39193


In [177]:
df_in_fdr_uniprot_species = remove_all_once_appearing_acc(df_in_fdr_uniprot_species)
df_in_fdr_uniprot_family = remove_all_once_appearing_acc(df_in_fdr_uniprot_family)
#short acc to acc between |
df_in_fdr_uniprot_species.Protein = df_in_fdr_uniprot_species.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})
df_in_fdr_uniprot_family.Protein = df_in_fdr_uniprot_family.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})

41809
all_accs:  263719
acc_to_keep  173483
accs_to_remove:  90236


/tmp/ipykernel_2935/3986558813.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(lambda acc_set: acc_set.intersection(acc_to_keep))


41424
all_accs:  717441
acc_to_keep  444185
accs_to_remove:  273256


In [232]:
# bacillus subtilis spec:1423 fam: 186817 ohne nur einmal auftretene Accs
describe_difference_between_species_and_family_identification_with_removed_accs("bacillus subtilis", 1423, 186817, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)


bacillus subtilis
37510
564
Anzahl Protein-Identifikationen auf Family Level für taxon 186817
    mit 1 identifikation:0
    mit 2 identifikationen: 94,
    mit 3 identifikationen: 45, 
    mit mehr als 3 Identifikationen: 150
Anzahl Protein-Identifikationen auf Species Level für taxon 1423
    mit 1 identifikation:0
    mit 2 identifikationen: 153,
    mit 3 identifikationen: 97, 
    mit mehr als 3 Identifikationen: 251
identified spectra species for taxon 1423: 738
    identified spectra family for taxon 186817: 561
identified accs species for taxon 1423: 502
    identified accs family for taxon 186817: 290

Eigenschaften der neu identifizierten Spektra:
all new identified spectra family: 0
identified exclusive for taxon 186817: 0
    not exclusive identified for taxon 186817, taxid to PSM number dict: 
    {}
Anzal Spekteb bereits vorher für andere Spezien identifiziert: 0


In [254]:
describe_difference_between_species_and_family_identification_with_removed_accs("staphylococcus aureus", 1280, 90964, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
#staphylococcus aureus


staphylococcus aureus
Anzahl Protein-Identifikationen auf Family Level für taxon 90964
    mit 1 identifikation:0
    mit 2 identifikationen: 94,
    mit 3 identifikationen: 45, 
    mit mehr als 3 Identifikationen: 150
Anzahl Protein-Identifikationen auf Species Level für taxon 1280
    mit 1 identifikation:0
    mit 2 identifikationen: 153,
    mit 3 identifikationen: 97, 
    mit mehr als 3 Identifikationen: 251
identified spectra species for taxon 1280: 738
    identified spectra family for taxon 90964: 561
identified accs species for taxon 1280: 502
    identified accs family for taxon 90964: 290

Eigenschaften der neu identifizierten Spektra:
all new identified spectra family: 0
identified exclusive for taxon 90964: 0
    not exclusive identified for taxon 90964, taxid to PSM number dict: 
    {}
Anzal Spekteb bereits vorher für andere Spezien identifiziert: 0


In [255]:
describe_difference_between_species_and_family_identification_with_removed_accs("Paracoccus denitrificans", 266, 31989, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
# Paracoccus denitrificans


Paracoccus denitrificans
Anzahl Protein-Identifikationen auf Family Level für taxon 31989
    mit 1 identifikation:0
    mit 2 identifikationen: 94,
    mit 3 identifikationen: 45, 
    mit mehr als 3 Identifikationen: 150
Anzahl Protein-Identifikationen auf Species Level für taxon 266
    mit 1 identifikation:0
    mit 2 identifikationen: 153,
    mit 3 identifikationen: 97, 
    mit mehr als 3 Identifikationen: 251
identified spectra species for taxon 266: 738
    identified spectra family for taxon 31989: 561
identified accs species for taxon 266: 502
    identified accs family for taxon 31989: 290

Eigenschaften der neu identifizierten Spektra:
all new identified spectra family: 0
identified exclusive for taxon 31989: 0
    not exclusive identified for taxon 31989, taxid to PSM number dict: 
    {}
Anzal Spekteb bereits vorher für andere Spezien identifiziert: 0


In [256]:
describe_difference_between_species_and_family_identification_with_removed_accs("chromobacterium violaceum", 536, 1499392, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
#chromobacterium violaceum

chromobacterium violaceum
Anzahl Protein-Identifikationen auf Family Level für taxon 1499392
    mit 1 identifikation:0
    mit 2 identifikationen: 94,
    mit 3 identifikationen: 45, 
    mit mehr als 3 Identifikationen: 150
Anzahl Protein-Identifikationen auf Species Level für taxon 536
    mit 1 identifikation:0
    mit 2 identifikationen: 153,
    mit 3 identifikationen: 97, 
    mit mehr als 3 Identifikationen: 251
identified spectra species for taxon 536: 738
    identified spectra family for taxon 1499392: 561
identified accs species for taxon 536: 502
    identified accs family for taxon 1499392: 290

Eigenschaften der neu identifizierten Spektra:
all new identified spectra family: 0
identified exclusive for taxon 1499392: 0
    not exclusive identified for taxon 1499392, taxid to PSM number dict: 
    {}
Anzal Spekteb bereits vorher für andere Spezien identifiziert: 0


In [257]:
describe_difference_between_species_and_family_identification_with_removed_accs("altermonas macleodii", 28108, 72275, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
#altermonas macleodii

altermonas macleodii
Anzahl Protein-Identifikationen auf Family Level für taxon 72275
    mit 1 identifikation:0
    mit 2 identifikationen: 94,
    mit 3 identifikationen: 45, 
    mit mehr als 3 Identifikationen: 150
Anzahl Protein-Identifikationen auf Species Level für taxon 28108
    mit 1 identifikation:0
    mit 2 identifikationen: 153,
    mit 3 identifikationen: 97, 
    mit mehr als 3 Identifikationen: 251
identified spectra species for taxon 28108: 738
    identified spectra family for taxon 72275: 561
identified accs species for taxon 28108: 502
    identified accs family for taxon 72275: 290

Eigenschaften der neu identifizierten Spektra:
all new identified spectra family: 0
identified exclusive for taxon 72275: 0
    not exclusive identified for taxon 72275, taxid to PSM number dict: 
    {}
Anzal Spekteb bereits vorher für andere Spezien identifiziert: 0


In [258]:
describe_difference_between_species_and_family_identification_with_removed_accs("paraburkholderia", [36873, 119219], 119060, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
#paraburkholderia

paraburkholderia
Anzahl Protein-Identifikationen auf Family Level für taxon 119060
    mit 1 identifikation:0
    mit 2 identifikationen: 94,
    mit 3 identifikationen: 45, 
    mit mehr als 3 Identifikationen: 150
Anzahl Protein-Identifikationen auf Species Level für taxon [36873, 119219]
    mit 1 identifikation:0
    mit 2 identifikationen: 404,
    mit 3 identifikationen: 249, 
    mit mehr als 3 Identifikationen: 1103
identified spectra species for taxon [36873, 119219]: 7304
    identified spectra family for taxon 119060: 561
identified accs species for taxon [36873, 119219]: 1757
    identified accs family for taxon 119060: 290

Eigenschaften der neu identifizierten Spektra:
all new identified spectra family: 507
identified exclusive for taxon 119060: 0
    not exclusive identified for taxon 119060, taxid to PSM number dict: 
    {186817: 510, 135621: 172, 90964: 331, 31989: 84, 119060: 48, 543: 345, 1499392: 94, 10699: 2, 32033: 36, 72275: 53, 82115: 35, 3051: 14, 188786: 10, 

In [301]:
# loading 9MM species and family:
fdr=0.05
tanca_species_df = get_df_in_fdr_without_decoy("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv/9MM_FASP_uniprot_Tanca_species_nr.t.xml_new_reduced.tsv", fdr, columns=['taxID_species'])
tanca_family_df = get_df_in_fdr_without_decoy("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv/9MM_FASP_uniprot_Tanca_family_nr.t.xml_new_reduced.tsv", fdr, columns=['taxID_family'])

In [302]:
tanca_species_df = remove_all_once_appearing_accs(tanca_species_df)
tanca_family_df = remove_all_once_appearing_accs(tanca_family_df)
#short acc to acc between |
tanca_species_df.Protein = tanca_species_df.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})
tanca_family_df.Protein = tanca_family_df.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})

acc_to_keep  9893
accs_to_remove:  5701


/home/jules/.local/lib/python3.9/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


acc_to_keep  66207
accs_to_remove:  31295


In [ ]:
describe_difference_between_species_and_family_identification_with_removed_accs("rhodotorula glutinis", 5535, 1799696, tanca_species_df, tanca_family_df)
# rhodotorula glutinis
# identified spectra species: 15 identified spectra family: 495

In [303]:
taxids_species = 5535
taxid_family = 1799696
    #load accs of taxon {'A0A3A5I3L3', 'A0A4T2HSI9', 'A0A4R6HX40', 'G9LQ63', 'A0A410ZMS7' ...}
species_taxa_specific_accs = get_taxa_specific_accs(taxids_species)
family_taxa_specific_accs = get_taxa_specific_accs(taxid_family)
print(len(species_taxa_specific_accs))
"A0A126QE45" in species_taxa_specific_accs    

47


True

In [270]:
# nach taxa
df_in_fdr_species_tax_tax = tanca_species_df.copy(deep=True)
df_in_fdr_family_tax_tax = tanca_family_df.copy(deep=True)
df_in_fdr_species_tax_tax = df_in_fdr_species_tax_tax[get_taxa_rows(df_in_fdr_species_tax_tax.taxID_species, 5535)]
df_in_fdr_species_tax_tax

,Unnamed: 0,Title,Peptide,Hyperscore,Protein,decoy,taxID,taxID_species
586,586,File9527 Spectrum25870 scans: 33671,SVEMHHEQLEAGLPGDNVGFNVK,53.8,{A0A126QE45},{False},{5535},{5535}
2384,2384,File9527 Spectrum5327 scans: 11074,TVDGPSAKDWR,39.9,"{A0A1E3XJ40, V4NDF0, A0A3L0YH61, L0LB70, A0A3S...",{False},"{1331042, 1032868, 1304873, 747, 562, 5535}","{562, 747, 5535}"
2555,2555,File9527 Spectrum15842 scans: 22641,IGGIGTVPVGR,39.2,"{A0A126QE45, B3LK50, I3Y0Y3, S5RW04, E9P9G3, A...",{False},"{562, 4932, 285006, 5535}","{562, 4932, 5535}"
2574,2574,File9527 Spectrum15923 scans: 22730,IGGIGTVPVGR,39.1,"{A0A126QE45, B3LK50, I3Y0Y3, S5RW04, E9P9G3, A...",{False},"{562, 4932, 285006, 5535}","{562, 4932, 5535}"
2575,2575,File9527 Spectrum15762 scans: 22553,IGGIGTVPVGR,39.1,"{A0A126QE45, B3LK50, I3Y0Y3, S5RW04, E9P9G3, A...",{False},"{562, 4932, 285006, 5535}","{562, 4932, 5535}"
2597,2597,File9527 Spectrum15689 scans: 22472,IGGIGTVPVGR,39.0,"{A0A126QE45, B3LK50, I3Y0Y3, S5RW04, E9P9G3, A...",{False},"{562, 4932, 285006, 5535}","{562, 4932, 5535}"
3214,3214,File9527 Spectrum22306 scans: 29751,IGGIGTVPVGRVETG,36.4,"{A0A126QE45, B3LK50, I3Y0Y3, S5RW04, E9P9G3, A...",{False},"{562, 4932, 285006, 5535}","{562, 4932, 5535}"
3402,3402,File9527 Spectrum5313 scans: 11059,TVDGPSAKDWR,35.7,"{A0A1E3XJ40, V4NDF0, A0A3L0YH61, L0LB70, A0A3S...",{False},"{1331042, 1032868, 1304873, 747, 562, 5535}","{562, 747, 5535}"
4056,4056,File9527 Spectrum5577 scans: 11349,TVDGPSAKDWR,33.5,"{A0A1E3XJ40, V4NDF0, A0A3L0YH61, L0LB70, A0A3S...",{False},"{1331042, 1032868, 1304873, 747, 562, 5535}","{562, 747, 5535}"
4123,4123,File9527 Spectrum5393 scans: 11147,TVDGPSAKDWR,33.3,"{A0A1E3XJ40, V4NDF0, A0A3L0YH61, L0LB70, A0A3S...",{False},"{1331042, 1032868, 1304873, 747, 562, 5535}","{562, 747, 5535}"


In [304]:
# keep only specific accs
#tanca_species_df, tanca_family_df
df_in_fdr_species_tax_acc = tanca_species_df.copy(deep=True)
df_in_fdr_family_tax_acc = tanca_family_df.copy(deep=True)
print(df_in_fdr_species_tax_acc.head(5))
print(df_in_fdr_family_tax_acc.head(5))
df_in_fdr_species_tax_acc.Protein = df_in_fdr_species_tax_acc.Protein.apply(lambda acc_set: \
                                                        acc_set.intersection(species_taxa_specific_accs))
df_in_fdr_family_tax_acc.Protein = df_in_fdr_family_tax_acc.Protein.apply(lambda acc_set: \
                                                        acc_set.intersection(family_taxa_specific_accs))
print(df_in_fdr_species_tax_acc.head(5))
print(df_in_fdr_family_tax_acc.head(5))
#remove rows with no accs
df_in_fdr_species_tax_acc = df_in_fdr_species_tax_acc[df_in_fdr_species_tax_acc.Protein != set()]
df_in_fdr_family_tax_acc = df_in_fdr_family_tax_acc[df_in_fdr_family_tax_acc.Protein != set()]
print(df_in_fdr_species_tax_acc.head(10))
print(df_in_fdr_family_tax_acc.head(10))


   Unnamed: 0                                Title  \
0           0  File9527 Spectrum33502 scans: 42067   
1           1   File9527 Spectrum9649 scans: 15828   
2           2   File9527 Spectrum9571 scans: 15742   
3           3  File9527 Spectrum33544 scans: 42113   
4           4  File9527 Spectrum20563 scans: 27834   

                                    Peptide  Hyperscore  \
0          GLEMASKNAQDGISLIQTAEGALTETHSILQR       108.9   
1  GGSGGSHGGGSGFGGESGGSYGGGEEASGSGGGYGGGSGK       105.0   
2  GGSGGSHGGGSGFGGESGGSYGGGEEASGSGGGYGGGSGK       100.3   
3          GLEMASKNAQDGISLIQTAEGALTETHSILQR        96.1   
4                 VAMISYSTGTSGSGADVEKVKEATR        91.3   

                                             Protein    decoy  \
0  {A0A0F6Y0X2, A0A0F7EJP9, A0A075R7M4, A0A502KJ7...  {False}   
1                                       {K1C9_HUMAN}  {False}   
2                                       {K1C9_HUMAN}  {False}   
3  {A0A0F6Y0X2, A0A0F7EJP9, A0A075R7M4, A0A502KJ7...  {False

In [305]:
# all accs
accs_spec = sorted([item for sublist in df_in_fdr_species_tax_acc.Protein for item in sublist])
accs_fam = sorted([item for sublist in df_in_fdr_family_tax_acc.Protein for item in sublist])

print(f"identified spectra species for taxon {taxids_species}: {len(set(df_in_fdr_species_tax_acc.Title))}\n\
identified spectra family for taxon {taxid_family}: {len(set(df_in_fdr_family_tax_acc.Title))}")
print(f"identified accs species for taxon {taxids_species}: {len(set(accs_spec))}\n\
identified accs family for taxon {taxid_family}: {len(set(accs_fam))}")

#count    
count_dict_spec = count_accs(accs_spec)
print(f"Anzahl Protein-Identifikationen auf Species Level für taxon {taxids_species}\n\
mit 1 identifikation:{count_dict_spec[1]}\n\
mit 2 identifikationen: {count_dict_spec[2]},\n\
mit 3 identifikationen: {count_dict_spec[3]}, \n\
mit mehr als 3 Identifikationen: {count_dict_spec['>3']}")     

count_dict_fam = count_accs(accs_fam)
print(f"Anzahl Protein-Identifikationen auf Family Level für taxon {taxid_family}\n\
mit 1 identifikation:{count_dict_fam[1]}\n\
mit 2 identifikationen: {count_dict_fam[2]},\n\
mit 3 identifikationen: {count_dict_fam[3]}, \n\
mit mehr als 3 Identifikationen: {count_dict_fam['>3']}") 

print()
print("Eigenschaften der neu identifizierten Spektra:")
df_family_tax_fam_not_in_species_df=df_in_fdr_family_tax_acc[~df_in_fdr_family_tax_acc.Title.isin(df_in_fdr_species_tax_acc.Title)]
print(f"all new identified spectra family: {len(set(df_family_tax_fam_not_in_species_df.Title))}")
df_only_one_taxa_tax_fam = df_family_tax_fam_not_in_species_df[df_family_tax_fam_not_in_species_df.taxID_family == {taxid_family}]
df_only_one_taxa_tax_fam = df_family_tax_fam_not_in_species_df[df_family_tax_fam_not_in_species_df.taxID_family == {taxid_family}]
taxa_count_dict_tax_fam = count_taxa(df_family_tax_fam_not_in_species_df.taxID_family)
print(f"identified exclusive for taxon {taxid_family}: {len(set(df_only_one_taxa_tax_fam.Title))}\n\
not exclusive identified for taxon {taxid_family}, taxid to PSM number dict: \n\
{taxa_count_dict_tax_fam}")
df_family_tax_fam_in_species_df = df_family_tax_fam_not_in_species_df[df_family_tax_fam_not_in_species_df.Title.isin(df_in_fdr_uniprot_species.Title)]
print(f"Anzal Spekteb bereits vorher für andere Spezien identifiziert: {len(set(df_family_tax_fam_in_species_df.Title))}")

identified spectra species for taxon 5535: 15
identified spectra family for taxon 1799696: 417
identified accs species for taxon 5535: 2
identified accs family for taxon 1799696: 448
Anzahl Protein-Identifikationen auf Species Level für taxon 5535
mit 1 identifikation:0
mit 2 identifikationen: 0,
mit 3 identifikationen: 0, 
mit mehr als 3 Identifikationen: 1
Anzahl Protein-Identifikationen auf Family Level für taxon 1799696
mit 1 identifikation:0
mit 2 identifikationen: 167,
mit 3 identifikationen: 62, 
mit mehr als 3 Identifikationen: 218

Eigenschaften der neu identifizierten Spektra:
all new identified spectra family: 404
identified exclusive for taxon 1799696: 235
not exclusive identified for taxon 1799696, taxid to PSM number dict: 
{1799696: 416, 4893: 138, 186822: 59, 543: 52, 33958: 42, 712: 36, 81852: 47, 'DECOY': 11}
Anzal Spekteb bereits vorher für andere Spezien identifiziert: 0


In [ ]:
# ohne remove

In [ ]:
df_in_fdr_uniprot_species = get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['species'], fdr, columns=['taxID_species']) 
df_in_fdr_uniprot_family =  get_df_in_fdr_without_decoy(uniprot_nr_reduced_tsv['family'], fdr, columns=['taxID_family'])
df_in_fdr_uniprot_species.Protein = df_in_fdr_uniprot_species.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})
df_in_fdr_uniprot_family.Protein = df_in_fdr_uniprot_family.Protein.apply(lambda acc_set: {acc.split('|')[1] \
                                                                             for acc in acc_set})

In [ ]:
describe_difference_between_species_and_family_identification_with_removed_accs("bacillus subtilis", 1423, 186817, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)

In [ ]:
# bacillus subtilis spec:1423 fam: 186817
# welche spectra komplett neu, Score Werte von 101.1 - 28.3, mean=33.8, median=30.5, davon 8 ribosomale Proteine,
# 100 enzyme, 29 uncharacterized, 3 Flagellar (von 285), ATP: 48
# anzahl unterschiedlicher Peptide von 1816 spectra, 846 unterschiediche Peptide und 466753 neue accs
# davon 450 speziell nur für bacillus, 904 auch in 543, 617 in 125621, 453 un 90964
df_family_186817_not_in_species_df, df_exclusive_new_psms_186817 = describe_difference_between_species_and_family_identification("bacillus subtilis", 1423, 186817, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)

#bacillus subtilis
#identified spectra species: 855 identified spectra family: 2352
#new identified spectra family: 1712
#identified only for taxon: 429
# all spectra counts per taxa: {186817: 1816, 72275: 405, 90964: 452, 543: 904, 32033: 545, 31989: 434, 135621: 617, 119060: 440, 1499392: 232, 206379: 199, 82115: 189, 3051: 11, 188786: 21, 194924: 48, 10662: 4, 10699: 6, 1033997: 1, 'DECOY': 6}
#davon wurde bereits vorher für andere spezien identifiziert: 1491 spektren.


get_new_spectra_characteristics(186817, df_family_186817_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

#Number of new identified accs: 4459
#mit 1 identifikation: 2793, mit 2 identifikationen: 543,
 #   mit 3 identifikationen: 496, mit 4 identifikationen: 97, 
  #  mit mehr als 4 identifikationen: 530
#In species df identified PSM with count_1: 412
#In species df identified PSM with count_2: 322
#In species df identified PSM with count_3: 168
#In species df identified PSM with count_4: 105
#In species df identified PSM with count_5: 1381

print('spectra characteristica for exclusive spectra: ')
get_new_spectra_characteristics(186817, df_exclusive_new_psms_186817, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

exclusive_psms_of_family_level_in_species(df_exclusive_new_psms_186817, df_in_fdr_uniprot_species, 1423)

set_accs = {acc for acc_set in df_exclusive_new_psms_186817.Protein for acc in acc_set}
print('Anzahl der neu identifizerten Accs, der exklusiven Spektra ', len(set_accs))
result_dict = count_psm_accs(df_in_fdr_uniprot_family, set_accs)
print(f"Accs mit einem PSM: {result_dict[1]}, Accs mit 2 PSM: {result_dict[2]}, Accs mit 3 PSM: {result_dict[3]}, Accs mit mehr als 4 PSM: {result_dict['>4']}")

# Anzahl der mehr als 2x identifizierten Accs, die noch nicht mehr als 2x auf Species Level identifiziert wurden
# set_all_family_accs_186817, set_all_species_accs_1423, acc_more_than_2_family_186817, acc_more_than_2_species_1423 = get_accs_with_more_than_2_psms(186817, 1423, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

#print(f"Accs family level mit mehr als 2 PSMs: {len(acc_more_than_2_family_186817)}, insgesamt: {len(set_all_family_accs_186817)}\
# weniger 2: {len(set_all_family_accs_186817)-len(acc_more_than_2_family_186817)}")


In [89]:
get_species_spectra_characteristics(1423, df_in_fdr_uniprot_species)

Number of identified accs on species level for taxon1423: 967
Anzahl Protein-Identifikationen auf Species Level für taxon 1423 accs 
     mit 1 identifikation:465, 
 mit 2 identifikationen: 153,
    mit 3 identifikationen: 97, 
 mit mehr als 3 identifikationen: 252
spectra characteristica for exclusive spectra: 
Number of identified accs on species level for exclusive taxon1423: 559
Anzahl Protein-Identifikationen auf Species Level für taxon 1423 accs 
     mit 1 identifikation:283, 
 mit 2 identifikationen: 93,
    mit 3 identifikationen: 65, 
 mit mehr als 3 identifikationen: 118


In [51]:
# staphylococcus aureus: spec: 1280 fam: 90964
df_family_90964_not_in_species_df, df_exclusive_new_psms_90964 = describe_difference_between_species_and_family_identification("staphylococcus aureus", 1280, 90964, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
#staphylococcus aureus
#identified spectra species: 928 identified spectra family: 2850
#new identified spectra family: 2203
#identified only for taxon: 46
# all spectra counts per taxa: {90964: 2217, 543: 2114, 186817: 560, 72275: 423, 32033: 314, 82115: 117, 31989: 182, 135621: 465, 1499392: 198, 194924: 26, 119060: 161, 10744: 13, 3051: 16, 206379: 89, 10662: 9, 10699: 20, 188786: 34, 'DECOY': 3}
#davon wurde bereits vorher für andere spezien identifiziert: 2162 spektren.
get_new_spectra_characteristics(90964, df_family_90964_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)
#Number of new identified accs: 2564
#mit 1 identifikation: 1338, mit 2 identifikationen: 689,
#    mit 3 identifikationen: 274, mit 4 identifikationen: 7, 
#    mit mehr als 4 identifikationen: 256
#In species df identified PSM with count_1: 72
#In species df identified PSM with count_2: 62
#In species df identified PSM with count_3: 94
#In species df identified PSM with count_4: 68
#In species df identified PSM with count_5: 837
print('spectra characteristica for exclusive spectra: ')
get_new_spectra_characteristics(90964, df_exclusive_new_psms_90964, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

#Number of new identified accs on family level for taxon90964: 71
#Anzahl Protein-Identifikationen auf Family Level für die mit taxon 90964 neu hinzugekommen accs     
# mit 1 identifikation:59, mit 2 identifikationen: 5,
#    mit 3 identifikationen: 5, mit 4 identifikationen: 1, 
#    mit mehr als 4 identifikationen: 1
#Anzahl der identifizierten Spektren, welche bereits auf species Level durch Proteine identifiziert wurden, welche auch in den neuen Identifikationen enthalten sind.
#Proteine die auf species Level 0 mal identifiziert wurden: 0.
#Proteine die auf species Level 1 mal identifiziert wurden: 15.
#Proteine die auf species Level 2 mal identifiziert wurden:: 31.
#Proteine die auf species Level 3 mal identifiziert wurden:: 37.
#Proteine die auf species Level 4 mal identifiziert wurden:: 20.
#Proteine die auf species Level mehr als 4 mal identifiziert wurden:: 0.
#Anzahl der neu identifizerten Accs, der exklusiven Spektra  71

exclusive_psms_of_family_level_in_species(df_exclusive_new_psms_90964, df_in_fdr_uniprot_species, 1280)


set_accs = {acc for acc_set in df_exclusive_new_psms_90964.Protein for acc in acc_set}
print('Anzahl der neu identifizerten Accs, der exklusiven Spektra ', len(set_accs))
result_dict = count_psm_accs(df_in_fdr_uniprot_family, set_accs)
print(f"Accs mit einem PSM: {result_dict[1]}, Accs mit 2 PSM: {result_dict[2]}, Accs mit 3 PSM: {result_dict[3]}, Accs mit mehr als 4 PSM: {result_dict['>4']}")
#Anzahl exclusiver 1280 PSMs family level: 46,     davon schon in species df für 1280 identifiziert: 0
#Anzahl der neu identifizerten Accs, der exklusiven Spektra  71
#Accs mit einem PSM: 51, Accs mit 2 PSM: 8, Accs mit 3 PSM: 3, Accs mit mehr als 4 PSM: 9

# Anzahl der mehr als 2x identifizierten Accs, die noch nicht mehr als 2x auf Species Level identifiziert wurden
set_all_family_accs_90964, set_all_species_accs_1280, acc_more_than_2_family_90964, acc_more_than_2_species_1280 = get_accs_with_more_than_2_psms(90964, 1280, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

print(f"Accs family level mit mehr als 2 PSMs: {len(acc_more_than_2_family_90964)}, insgesamt: {len(set_all_family_accs_90964)}\
 weniger 2: {len(set_all_family_accs_90964)-len(acc_more_than_2_family_90964)}")

# wieviel exclusive für SA on spezies level)
print("exclusive spectra für staphylococcus aureus on spezies level: ", len(set(df_in_fdr_uniprot_species[df_in_fdr_uniprot_species.taxID_species=={1280}].Title)))

staphylococcus aureus
identified spectra species for taxon 1280: 928           
 identified spectra family for taxon 90964: 2850
all new identified spectra family  2203
identified exclusive for taxon 90964: 46
     not exclusive identified for taxon 90964 taxid to PSM number dict: {90964: 2217, 543: 2114, 186817: 560, 72275: 423, 32033: 314, 82115: 117, 31989: 182, 135621: 465, 1499392: 198, 194924: 26, 119060: 161, 10744: 13, 3051: 16, 206379: 89, 10662: 9, 10699: 20, 188786: 34, 'DECOY': 3}
Davon wurde bereits vorher für andere spezien identifiziert: 2162 spektren.
Number of new identified accs on family level for taxon90964: 548
Anzahl Protein-Identifikationen auf Family Level für die mit taxon 90964 neu hinzugekommen accs     
 mit 1 identifikation:246, mit 2 identifikationen: 90,
    mit 3 identifikationen: 57, mit 4 identifikationen: 34, 
    mit mehr als 4 identifikationen: 121
Anzahl der identifizierten Spektren, welche bereits auf species Level durch Proteine identifiziert wur

In [73]:
get_species_spectra_characteristics(1280, df_in_fdr_uniprot_species)

Number of identified accs on species level for taxon1280: 2797
Anzahl Protein-Identifikationen auf Species Level für taxon 1280 accs 
     mit 1 identifikation:1209, 
 mit 2 identifikationen: 380,
    mit 3 identifikationen: 327, 
 mit mehr als 3 identifikationen: 881
spectra characteristica for exclusive spectra: 
Number of identified accs on species level for exclusive taxon1280: 1427
Anzahl Protein-Identifikationen auf Species Level für taxon 1280 accs 
     mit 1 identifikation:745, 
 mit 2 identifikationen: 240,
    mit 3 identifikationen: 140, 
 mit mehr als 3 identifikationen: 302


In [52]:

#Alle neuen PSM davon alle mit nur einem PSM/Protein entfernen, wie viele bleiben übrig?
#Von exklusiven PSMs, wie viele Proteine nur mit einem PSM? Wie viele schon auf spezies ebene identifiziert?

In [53]:
# Paracoccus denitrificans 266, 31989
df_family_31989_not_in_species_df, df_exclusive_new_psms_31989 = describe_difference_between_species_and_family_identification("Paracoccus denitrificans", 266, 31989, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
# Paracoccus denitrificans
#identified spectra species: 614 identified spectra family: 3026
#new identified spectra family: 2583
#identified only for taxon: 975
# all spectra counts per taxa: {82115: 734, 31989: 2895, 1499392: 469, 186817: 451, 135621: 1042, 90964: 387, 119060: 746, 543: 776, 32033: 603, 72275: 523, 194924: 141, 206379: 255, 10744: 4, 3051: 26, 10662: 6, 188786: 19, 10699: 10, 1033997: 3, 'DECOY': 11}
#davon wurde bereits vorher für andere spezien identifiziert: 1963 spektren.
get_new_spectra_characteristics(31989, df_family_31989_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)
# Number of new identified accs: 11958
#mit 1 identifikation: 7454, mit 2 identifikationen: 1968,
#    mit 3 identifikationen: 824, mit 4 identifikationen: 438, 
#    mit mehr als 4 identifikationen: 1274
#In species df identified PSM with count_1: 477
#In species df identified PSM with count_2: 186
#In species df identified PSM with count_3: 143
#In species df identified PSM with count_4: 114
#In species df identified PSM with count_5: 898
print('spectra characteristica for exclusive spectra: ')
get_new_spectra_characteristics(31989, df_exclusive_new_psms_31989, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

exclusive_psms_of_family_level_in_species(df_exclusive_new_psms_31989, df_in_fdr_uniprot_species, 266)

set_accs = {acc for acc_set in df_exclusive_new_psms_31989.Protein for acc in acc_set}
print('Anzahl der neu identifizerten Accs, der exklusiven Spektra ', len(set_accs))
result_dict = count_psm_accs(df_in_fdr_uniprot_family, set_accs)
print(f"Accs mit einem PSM: {result_dict[1]}, Accs mit 2 PSM: {result_dict[2]}, Accs mit 3 PSM: {result_dict[3]}, Accs mit mehr als 4 PSM: {result_dict['>4']}")


Paracoccus denitrificans
identified spectra species for taxon 266: 614           
 identified spectra family for taxon 31989: 3026
all new identified spectra family  2583
identified exclusive for taxon 31989: 975
     not exclusive identified for taxon 31989 taxid to PSM number dict: {82115: 734, 31989: 2895, 1499392: 469, 186817: 451, 135621: 1042, 90964: 387, 119060: 746, 543: 776, 32033: 603, 72275: 523, 194924: 141, 206379: 255, 10744: 4, 3051: 26, 10662: 6, 188786: 19, 10699: 10, 1033997: 3, 'DECOY': 11}
Davon wurde bereits vorher für andere spezien identifiziert: 1963 spektren.
Number of new identified accs on family level for taxon31989: 13016
Anzahl Protein-Identifikationen auf Family Level für die mit taxon 31989 neu hinzugekommen accs     
 mit 1 identifikation:7530, mit 2 identifikationen: 2967,
    mit 3 identifikationen: 585, mit 4 identifikationen: 643, 
    mit mehr als 4 identifikationen: 1291
Anzahl der identifizierten Spektren, welche bereits auf species Level durch P

In [74]:
get_species_spectra_characteristics(266, df_in_fdr_uniprot_species)

Number of identified accs on species level for taxon266: 239
Anzahl Protein-Identifikationen auf Species Level für taxon 266 accs 
     mit 1 identifikation:117, 
 mit 2 identifikationen: 45,
    mit 3 identifikationen: 16, 
 mit mehr als 3 identifikationen: 61
spectra characteristica for exclusive spectra: 
Number of identified accs on species level for exclusive taxon266: 138
Anzahl Protein-Identifikationen auf Species Level für taxon 266 accs 
     mit 1 identifikation:69, 
 mit 2 identifikationen: 28,
    mit 3 identifikationen: 14, 
 mit mehr als 3 identifikationen: 27


In [88]:
# chromobacterium violaceum spec: 536 fam: 1499392
df_family_1499392_not_in_species_df, df_exclusive_new_psms_1499392 = describe_difference_between_species_and_family_identification("chromobacterium violaceum", 536, 1499392, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
#chromobacterium violaceum
#identified spectra species: 1129 identified spectra family: 1534
#new identified spectra family: 677
#identified only for taxon: 95
# all spectra counts per taxa: {1499392: 715, 32033: 271, 186817: 165, 135621: 372, 72275: 193, 119060: 459, 31989: 186, 543: 246, 206379: 165, 90964: 83, 82115: 99, 194924: 46, 188786: 48, 3051: 4, 10699: 1, 'DECOY': 4, 10744: 1, 1033997: 1}
#davon wurde bereits vorher für andere spezien identifiziert: 649 spektren.
get_new_spectra_characteristics(1499392, df_family_1499392_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)
# Number of new identified accs: 1708
#mit 1 identifikation: 531, mit 2 identifikationen: 465,
#    mit 3 identifikationen: 405, mit 4 identifikationen: 278, 
#    mit mehr als 4 identifikationen: 29
#In species df identified PSM with count_1: 41
#In species df identified PSM with count_2: 70
#In species df identified PSM with count_3: 50
#In species df identified PSM with count_4: 23
#In species df identified PSM with count_5: 680
print('spectra characteristica for exclusive spectra: ')
get_new_spectra_characteristics(1499392, df_exclusive_new_psms_1499392, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

exclusive_psms_of_family_level_in_species(df_exclusive_new_psms_1499392, df_in_fdr_uniprot_species, 536)

set_accs = {acc for acc_set in df_exclusive_new_psms_1499392.Protein for acc in acc_set}
print('Anzahl der neu identifizerten Accs, der exklusiven Spektra ', len(set_accs))
result_dict = count_psm_accs(df_in_fdr_uniprot_family, set_accs)
print(f"Accs mit einem PSM: {result_dict[1]}, Accs mit 2 PSM: {result_dict[2]}, Accs mit 3 PSM: {result_dict[3]}, Accs mit mehr als 4 PSM: {result_dict['>4']}")


chromobacterium violaceum
identified spectra species for taxon 536: 1129           
 identified spectra family for taxon 1499392: 1534
all new identified spectra family  677
identified exclusive for taxon 1499392: 95
     not exclusive identified for taxon 1499392 taxid to PSM number dict: {1499392: 715, 32033: 271, 186817: 165, 135621: 372, 72275: 193, 119060: 459, 31989: 186, 543: 246, 206379: 165, 90964: 83, 82115: 99, 194924: 46, 188786: 48, 3051: 4, 10699: 1, 'DECOY': 4, 10744: 1, 1033997: 1}
Davon wurde bereits vorher für andere spezien identifiziert: 649 spektren.
Number of new identified accs on family level for taxon1499392: 972
Anzahl Protein-Identifikationen auf Family Level für die mit taxon 1499392 neu hinzugekommen accs     
 mit 1 identifikation:481, mit 2 identifikationen: 223,
    mit 3 identifikationen: 83, mit 4 identifikationen: 16, 
    mit mehr als 4 identifikationen: 169
Anzahl der identifizierten Spektren, welche bereits auf species Level durch Proteine identifi

In [75]:
get_species_spectra_characteristics(536, df_in_fdr_uniprot_species)

Number of identified accs on species level for taxon536: 794
Anzahl Protein-Identifikationen auf Species Level für taxon 536 accs 
     mit 1 identifikation:327, 
 mit 2 identifikationen: 146,
    mit 3 identifikationen: 98, 
 mit mehr als 3 identifikationen: 223
spectra characteristica for exclusive spectra: 
Number of identified accs on species level for exclusive taxon536: 389
Anzahl Protein-Identifikationen auf Species Level für taxon 536 accs 
     mit 1 identifikation:222, 
 mit 2 identifikationen: 63,
    mit 3 identifikationen: 26, 
 mit mehr als 3 identifikationen: 78


In [87]:
# altermonas macleodii spec: 28108, fam: 72275
df_family_72275_not_in_species_df, df_exclusive_new_psms_72275 = describe_difference_between_species_and_family_identification("altermonas macleodii", 28108, 72275, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
#altermonas macleodii
#identified spectra species: 895 identified spectra family: 2224
#new identified spectra family: 1562
#identified only for taxon: 293
# all spectra counts per taxa: {186817: 279, 72275: 1785, 90964: 361, 543: 1008, 135621: 785, 1499392: 213, 32033: 545, 206379: 188, 119060: 501, 31989: 363, 82115: 175, 3051: 25, 188786: 44, 10744: 11, 194924: 82, 10699: 6, 10662: 4, 'DECOY': 5, 1033997: 1}
#davon wurde bereits vorher für andere spezien identifiziert: 1432 spektren.
get_new_spectra_characteristics(72275, df_family_72275_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)
#mit 1 identifikation: 2137, mit 2 identifikationen: 320,
#    mit 3 identifikationen: 1153, mit 4 identifikationen: 82, 
#    mit mehr als 4 identifikationen: 1457
#In species df identified PSM with count_1: 133
#In species df identified PSM with count_2: 58
#In species df identified PSM with count_3: 112
#In species df identified PSM with count_4: 20
#In species df identified PSM with count_5: 833
print('spectra characteristica for exclusive spectra: ')
get_new_spectra_characteristics(72275, df_exclusive_new_psms_72275, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)

exclusive_psms_of_family_level_in_species(df_exclusive_new_psms_72275, df_in_fdr_uniprot_species, 28108)

set_accs = {acc for acc_set in df_exclusive_new_psms_72275.Protein for acc in acc_set}
print('Anzahl der neu identifizerten Accs, der exklusiven Spektra ', len(set_accs))
result_dict = count_psm_accs(df_in_fdr_uniprot_family, set_accs)
print(f"Accs mit einem PSM: {result_dict[1]}, Accs mit 2 PSM: {result_dict[2]}, Accs mit 3 PSM: {result_dict[3]}, Accs mit mehr als 4 PSM: {result_dict['>4']}")


altermonas macleodii
identified spectra species for taxon 28108: 895           
 identified spectra family for taxon 72275: 2224
all new identified spectra family  1562
identified exclusive for taxon 72275: 293
     not exclusive identified for taxon 72275 taxid to PSM number dict: {186817: 279, 72275: 1785, 90964: 361, 543: 1008, 135621: 785, 1499392: 213, 32033: 545, 206379: 188, 119060: 501, 31989: 363, 82115: 175, 3051: 25, 188786: 44, 10744: 11, 194924: 82, 10699: 6, 10662: 4, 'DECOY': 5, 1033997: 1}
Davon wurde bereits vorher für andere spezien identifiziert: 1432 spektren.
Number of new identified accs on family level for taxon72275: 6161
Anzahl Protein-Identifikationen auf Family Level für die mit taxon 72275 neu hinzugekommen accs     
 mit 1 identifikation:2929, mit 2 identifikationen: 1115,
    mit 3 identifikationen: 593, mit 4 identifikationen: 265, 
    mit mehr als 4 identifikationen: 1259
Anzahl der identifizierten Spektren, welche bereits auf species Level durch Protei

In [76]:
get_species_spectra_characteristics(28108, df_in_fdr_uniprot_species)
    

Number of identified accs on species level for taxon28108: 562
Anzahl Protein-Identifikationen auf Species Level für taxon 28108 accs 
     mit 1 identifikation:260, 
 mit 2 identifikationen: 92,
    mit 3 identifikationen: 41, 
 mit mehr als 3 identifikationen: 169
spectra characteristica for exclusive spectra: 
Number of identified accs on species level for exclusive taxon28108: 165
Anzahl Protein-Identifikationen auf Species Level für taxon 28108 accs 
     mit 1 identifikation:90, 
 mit 2 identifikationen: 21,
    mit 3 identifikationen: 15, 
 mit mehr als 3 identifikationen: 39


In [84]:
# paraburkholderia spec: 36873 + 119219, fam: 119060
df_family_36873_not_in_species_df, df_exclusive_new_psms_119060 = describe_difference_between_species_and_family_identification("paraburkholderia", [36873, 119219], 119060, df_in_fdr_uniprot_species, df_in_fdr_uniprot_family)
#paraburkholderia
#identified spectra species for taxon [36873, 119219]: 7708           
# identified spectra family for taxon 119060: 8330
#all new identified spectra family  1912
#identified exclusive for taxon 119060: 592
#     not exclusive identified for taxon 119060 taxid to PSM number dict: 
    #{119060: 2037, 135621: 1102, 543: 608, 32033: 495, 186817: 276, 90964: 215, 31989: 432, 72275: 342,  
    #206379: 143, 1499392: 231, 82115: 205, 194924: 89, 3051: 17, 188786: 29, 10699: 6, 'DECOY': 8, 10662: 8, 1033997: 2}
#Davon wurde bereits vorher für andere spezien identifiziert: 1463 spektren.

get_new_spectra_characteristics(119060, df_family_36873_not_in_species_df, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)
# Number of new identified accs on family level for taxon 119060: 5278
#Anzahl Protein-Identifikationen auf Family Level für die mit taxon 119060 neu hinzugekommen accs     
# mit 1 identifikation:2719, mit 2 identifikationen: 1165,
#    mit 3 identifikationen: 436, mit 4 identifikationen: 400, 
#    mit mehr als 4 identifikationen: 558
#Anzahl der identifizierten Spektren, welche bereits auf species Level durch Proteine identifiziert wurden, welche auch in den neuen Identifikationen enthalten sind.
#Proteine die auf species Level 0 mal identifiziert wurden: 0.
#Proteine die auf species Level 1 mal identifiziert wurden: 1107.
#Proteine die auf species Level 2 mal identifiziert wurden:: 585.
#Proteine die auf species Level 3 mal identifiziert wurden:: 537.
#Proteine die auf species Level 4 mal identifiziert wurden:: 587.

# exclusive spectra = # 615
print('spectra characteristica for exclusive spectra: ')
get_new_spectra_characteristics(119060, df_exclusive_new_psms_119060, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)
# Number of new identified accs on family level for taxon119060: 1149
# Anzahl Protein-Identifikationen auf Family Level für die mit taxon 119060 neu hinzugekommen accs     
# mit 1 identifikation:814, mit 2 identifikationen: 205,
#    mit 3 identifikationen: 84, mit 4 identifikationen: 14, 
#    mit mehr als 4 identifikationen: 32
#Anzahl der identifizierten Spektren, welche bereits auf species Level durch Proteine identifiziert wurden, 
#welche auch in den neuen Identifikationen enthalten sind.
#Proteine die auf species Level 0 mal identifiziert wurden: 0.
#Proteine die auf species Level 1 mal identifiziert wurden: 646.
#Proteine die auf species Level 2 mal identifiziert wurden:: 219.
#Proteine die auf species Level 3 mal identifiziert wurden:: 213.
#Proteine die auf species Level 4 mal identifiziert wurden:: 229.
#Proteine die auf species Level mehr als 4 mal identifiziert wurden:: 243.

exclusive_psms_of_family_level_in_species(df_exclusive_new_psms_119060, df_in_fdr_uniprot_species, [36873,119219])

set_accs = {acc for acc_set in df_exclusive_new_psms_119060.Protein for acc in acc_set}
print('Anzahl der neu identifizerten Accs, der exklusiven Spektra ', len(set_accs))
result_dict = count_psm_accs(df_in_fdr_uniprot_family, set_accs)
print(f"Accs mit einem PSM: {result_dict[1]}, Accs mit 2 PSM: {result_dict[2]}, Accs mit 3 PSM: {result_dict[3]}, Accs mit mehr als 4 PSM: {result_dict['>4']}")



paraburkholderia
identified spectra species for taxon [36873, 119219]: 7708           
 identified spectra family for taxon 119060: 8330
all new identified spectra family  1912
identified exclusive for taxon 119060: 592
     not exclusive identified for taxon 119060 taxid to PSM number dict: {119060: 2037, 135621: 1102, 543: 608, 32033: 495, 186817: 276, 90964: 215, 31989: 432, 72275: 342, 206379: 143, 1499392: 231, 82115: 205, 194924: 89, 3051: 17, 188786: 29, 10699: 6, 'DECOY': 8, 10662: 8, 1033997: 2}
Davon wurde bereits vorher für andere spezien identifiziert: 1463 spektren.
Number of new identified accs on family level for taxon119060: 5278
Anzahl Protein-Identifikationen auf Family Level für die mit taxon 119060 neu hinzugekommen accs     
 mit 1 identifikation:2719, mit 2 identifikationen: 1165,
    mit 3 identifikationen: 436, mit 4 identifikationen: 400, 
    mit mehr als 4 identifikationen: 558
Anzahl der identifizierten Spektren, welche bereits auf species Level durch Protei

In [85]:
get_species_spectra_characteristics([36873, 119219], df_in_fdr_uniprot_species)

Number of identified accs on species level for taxon[36873, 119219]: 2718
Anzahl Protein-Identifikationen auf Species Level für taxon [36873, 119219] accs 
     mit 1 identifikation:961, 
 mit 2 identifikationen: 405,
    mit 3 identifikationen: 249, 
 mit mehr als 3 identifikationen: 1103
spectra characteristica for exclusive spectra: 
Number of identified accs on species level for exclusive taxon[36873, 119219]: 2479
Anzahl Protein-Identifikationen auf Species Level für taxon [36873, 119219] accs 
     mit 1 identifikation:893, 
 mit 2 identifikationen: 391,
    mit 3 identifikationen: 245, 
 mit mehr als 3 identifikationen: 950


In [ ]:
# mind 3 PSMs pro Accs, wieviele neue PSMS

In [ ]:

############ compare additional identified reference to uniprot species ###############################


In [ ]:
#### differences beween DB ####

In [147]:
ref_db = list(SeqIO.parse("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/Kleiner_ref_db/Kleiner_ref_crap.fasta", "fasta"))

In [148]:
uni_spec_db = list(SeqIO.parse("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/Tax2Proteome_DB/kleiner_species_nr_crap.fasta", "fasta"))

In [120]:
def add_accs_with_no_diamond_result(ref_db, set_to_compare):
    missing_accs = set()
    for entry in ref_db:
        if entry.id not in set_to_compare:
            missing_accs.add(entry.id)
    return missing_accs
    
# file enthält alle kleiner accs die mit uniprot_species_nr DB Sequenzen gematched haben
acc_score_df = pd.read_csv('/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/number_ref_seq_in_db/matches_tax2proteome_kleiner_ref_one_accs.tsv', header=None, names=["acc", "score"], sep="\t")
set_kleiner_accs_100_match = set(acc_score_df[acc_score_df.score == 100].acc)
# add sequences with no match
set_kleiner_accs_not_100_match = set(acc_score_df[acc_score_df.score != 100].acc)
set_accs_identified_in_ref = {item for sublist in df_in_fdr_ref.Protein for item in sublist if 'REVERSED' not in item}
acc_no_blast_match = all_acc_in_ref_df.difference(set(acc_score_df.acc))
#print(acc_no_blast_match)
set_accs_missing_match = add_accs_with_no_diamond_result(ref_db, set(acc_score_df.acc))
print("Number of Proteins/sequences/accs in Kleiner Referenz DB with no match in uniprot species DB: ", len(set_accs_missing_match))
set_kleiner_accs_not_100_match = set_kleiner_accs_not_100_match.union(set_accs_missing_match)
print(f"Number of Proteins/sequences/accs in tax2proteome uniprot species db with 100% match in Kleiner Referenz DB: {len(set_kleiner_accs_100_match)}\n\
Number of Proteins/sequences/accs in Kleiner Referenz DB without 100% match zu uniprot species DB: {len(set_kleiner_accs_not_100_match)}.")


Number of Proteins/sequences/accs in Kleiner Referenz DB with no match in uniprot species DB:  1488
Number of Proteins/sequences/accs in tax2proteome uniprot species db with 100% match in Kleiner Referenz DB: 106305
Number of Proteins/sequences/accs in Kleiner Referenz DB without 100% match zu uniprot species DB: 16783.


In [125]:
# Properties Referenz DF
# df filtered by accs names
ref_df_acc_in_t2p_df = df_in_fdr_ref[get_acc_rows(df_in_fdr_ref.Protein, set_kleiner_accs_100_match, False)]
ref_df_acc_not_in_t2p_df = df_in_fdr_ref[get_not_acc_rows(df_in_fdr_ref.Protein, set_kleiner_accs_100_match, False)]
ref_df_acc_not_in_t2p_df_2 = df_in_fdr_ref[get_acc_rows(df_in_fdr_ref.Protein, set_kleiner_accs_not_100_match, True)]

#print(ref_df_acc_not_in_t2p_df[['Title', 'Protein', 'Hyperscore']].head())
#print(ref_df_acc_not_in_t2p_df_2[['Title', 'Protein', 'Hyperscore']].head())
print(f"Number identified spectra with Kleiner Referenz DB in fdr: {len(set(df_in_fdr_ref.Title))}")
print("Referenz DF filtered by accession names")
print(f"Number identified spectra with Kleiner Referenz DB, Accs auch in uniprot species DB enthalten: {len(set(ref_df_acc_in_t2p_df.Title))}")
print(f"Number identified spectra with Kleiner Referenz DB, Accs ohne 100% match zu uniprot species DB enthalten: {len(set(ref_df_acc_not_in_t2p_df.Title))}")
print(f"Number identified spectra of 100% match accs + < 100% match accs: {len(set(ref_df_acc_not_in_t2p_df.Title)) + len(set(ref_df_acc_in_t2p_df.Title))}")
# print(f"number identified spectra in ref in fdr with all acc in Protein set not 100% match to t2p DB: {len(set(ref_df_acc_not_in_t2p_df_2.Title))}")

Number identified spectra with Kleiner Referenz DB in fdr: 41365
Referenz DF filtered by accession names
Number identified spectra with Kleiner Referenz DB, Accs auch in uniprot species DB enthalten: 38645
Number identified spectra with Kleiner Referenz DB, Accs ohne 100% match zu uniprot species DB enthalten: 2767
Number identified spectra of 100% match accs + < 100% match accs: 41412


In [136]:
# Properties uniprot species DF

# Properties spectra between 20.8 - 23.7:
df_uni_spec_in_missing_th = df_full_uniprot_species[(df_full_uniprot_species.Hyperscore >= 20.8)  & (df_full_uniprot_species.Hyperscore<23.7) & (df_full_uniprot_species.decoy!={True})].sort_values(by=['Hyperscore'])
df_ref_low_score = df_in_fdr_ref[(df_in_fdr_ref.Hyperscore >= 20.8)  & (df_in_fdr_ref.Hyperscore<23.7) & (df_in_fdr_ref.decoy!={True})]
df_uni_spec_with_20_8_threshold = df_full_uniprot_species[(df_full_uniprot_species.Hyperscore >= 20.8) & (df_full_uniprot_species.decoy!={True})]
spectra_with_low_th = set(df_uni_spec_in_missing_th.Title)
peptides_with_low_th = set(df_uni_spec_in_missing_th.Peptide)

# sollte 0 sein
df_uni_spec_in_fdr_with_low_th_spectra =df_in_fdr_uniprot_species[df_in_fdr_uniprot_species.Title.isin(spectra_with_low_th)]
# Peptide können auch durch besser passende Spekten ientifiziert werden
df_uni_spec_in_fdr_with_low_th_peptide = df_in_fdr_uniprot_species[df_in_fdr_uniprot_species.Peptide.isin(peptides_with_low_th)]

print("Number identified SPECTRA with Reference DB in Hyperscore range betwenn 20.8 -23.7: ", len(set(df_ref_low_score.Title)))
print("Number identified SPECTRA with uniprot species DB in Hyperscore range betwenn 20.8 -23.7: ", len(spectra_with_low_th))
print("Number identified PEPTIDES with uniprot species DB in Hyperscore range betwenn 20.8 -23.7: ", len(peptides_with_low_th))
print("Anzahl SPECTRA, die in Hyperscore range zw. 20.8 -23.7 identifiziert wurden und auch über FDR Threshold: ", len(set(df_uni_spec_in_fdr_with_low_th_spectra.Title)))
print("Anzahl PEPTIDES, die in Hyperscore range zw. 20.8 -23.7 identifiziert wurden und auch über FDR Threshold: ", len(set(df_uni_spec_in_fdr_with_low_th_peptide.Peptide)))
print()
#Difference reference df and uniprot df with same th
ref_identified_peptides_not_in_t2p_uni_spec = set(df_in_fdr_ref.Peptide).difference(set(df_uni_spec_with_20_8_threshold.Peptide))

print('Number identified SPECTRA with uniprot species DF in FDR: ', len(set(df_in_fdr_uniprot_species.Title)))
print('Absolute number with reference DB more identified SPECTRA: ', len(set(df_in_fdr_ref.Title)) - len(set(df_in_fdr_uniprot_species.Title)))
print('Number identified SPECTRA with Reference DB not contained in uniprot species DB mit selbem Threshold: ', len(set(df_in_fdr_ref[~df_in_fdr_ref.Title.isin(df_uni_spec_with_20_8_threshold.Title)].Title)))
print('Number identified SPECTRA with Reference DB not contained in uniprot species DB mit FDR Threshold: ', len(set(df_in_fdr_ref[~df_in_fdr_ref.Title.isin(df_in_fdr_uniprot_species.Title)].Title)))

print(f"Number identified PEPTIDES with Referenz-DB, aber nicht von Uniprot DB identifiziert mit referenz Threshold: {len(ref_identified_peptides_not_in_t2p_uni_spec)}")
print('Number identified SPECTRA with Uniprot species DB mit selbem Threshold not contained in reference DB: ', len(set(df_uni_spec_with_20_8_threshold[~df_uni_spec_with_20_8_threshold.Title.isin(df_in_fdr_ref.Title)].Title)))
print('Number identified SPECTRA with Uniprot species DB mit FDR Threshold not contained in reference DB: ', len(set(df_in_fdr_uniprot_species[~df_in_fdr_uniprot_species.Title.isin(df_in_fdr_ref.Title)].Title)))


# search peptides not in uniprot_nr species in DB
#uniprot_identified_peptides_not_ref = set(df_uni_spec_ref_th.Peptide).difference(set(df_in_fdr_ref.Peptide))
#print(len(uniprot_identified_peptides_not_ref))
#ref_identified_peptides_not_uniprot_species
#print(peptides_in_this_range)
#print(df_uni_spec_in_missing_th.head())
#df_uni_spec_in_missing_th.tail()

Number identified SPECTRA with Reference DB in Hyperscore range betwenn 20.8 -23.7:  4542
Number identified SPECTRA with uniprot species DB in Hyperscore range betwenn 20.8 -23.7:  8392
Number identified PEPTIDES with uniprot species DB in Hyperscore range betwenn 20.8 -23.7:  7693
Anzahl SPECTRA, die in Hyperscore range zw. 20.8 -23.7 identifiziert wurden und auch über FDR Threshold:  0
Anzahl PEPTIDES, die in Hyperscore range zw. 20.8 -23.7 identifiziert wurden und auch über FDR Threshold:  1110

Number identified SPECTRA with uniprot species DF in FDR:  39193
Absolute number with reference DB more identified SPECTRA:  2172
Number identified SPECTRA with Reference DB not contained in uniprot species DB mit selbem Threshold:  1086
Number identified SPECTRA with Reference DB not contained in uniprot species DB mit FDR Threshold:  4830
Number identified PEPTIDES with Referenz-DB, aber nicht von Uniprot DB identifiziert mit referenz Threshold: 1501
Number identified SPECTRA with Uniprot 

In [54]:
#proteins_ = {item for sublist in df_acc_not_in_this_range.Protein for item in sublist}
#print(len(proteins_.intersection(proteins_in_this_range)))
# print('uniprot species df acc: ', len(set(df_acc_not_in_this_range.Title)))
# print('uniprot species df peptides: ',len(set(df_peptides_not_in_this_range.Title)))

In [144]:
set_spectra_in_uniprot_spec_in_fdr = set(df_in_fdr_uniprot_species.Title)
ref_df_spectra_not_in_uniprot_spec = df_in_fdr_ref[~df_in_fdr_ref.Title.isin(set_spectra_in_uniprot_spec_in_fdr)].sort_values(by=['Hyperscore'])
set_kleiner_accs_of_unidentified_spectra = {item for sublist in ref_df_spectra_not_in_uniprot_spec.Protein for item in sublist}
set_kleiner_accs_of_unidentified_spectra_23_th = {item for sublist in ref_df_spectra_not_in_uniprot_spec[ref_df_spectra_not_in_uniprot_spec.Hyperscore>=23.7].Protein for item in sublist}
#ref_df_accs_not_in_uniprot_spec = df_in_fdr_ref[get_not_spectra_rows(df_in_fdr_ref.Title, set_spectra_in_uniprot_spec_in_fdr)]
proteins_that_should_be_contained_in_t2P = set_kleiner_accs_of_unidentified_spectra_23_th.intersection(set_kleiner_accs_100_match)
ref_df_psms_that_should_be_contained_in_t2P = ref_df_spectra_not_in_uniprot_spec[get_acc_rows(ref_df_spectra_not_in_uniprot_spec.Protein, proteins_that_should_be_contained_in_t2P, True)]

print(f"Number SPECTRA not identified with uniprot species DB: {len(set(ref_df_spectra_not_in_uniprot_spec.Title))}")
print(f"davon number SPECTRA unter Hyperscore von 23.7: {len(set(ref_df_spectra_not_in_uniprot_spec[ref_df_spectra_not_in_uniprot_spec.Hyperscore<23.7].Title))}")
print(f"davon number SPECTRA mit Hyperscore >= 23.7: {len(set(ref_df_spectra_not_in_uniprot_spec[ref_df_spectra_not_in_uniprot_spec.Hyperscore>=23.7].Title))}")
print(f"Number ACCs associate to by uniprot_species DB unidentified spectra: {len(set_kleiner_accs_of_unidentified_spectra)}")
print(f"Number of unidentified ACCs with match < 100% with blast: {len(set_kleiner_accs_of_unidentified_spectra.intersection(set_kleiner_accs_not_100_match))}")
print(f"Number of unidentified ACCs with match < 100% with blast und Hyperscore >= 23.7: {len(set_kleiner_accs_of_unidentified_spectra_23_th.intersection(set_kleiner_accs_not_100_match))}")
print("Anzahl Peptide die in uniprot species DB enthalten sein sollten: ",len(set(ref_df_psms_that_should_be_contained_in_t2P.Peptide)))

print(f"Number of unidentified ACCs with match = 100% with blast: {len(set_kleiner_accs_of_unidentified_spectra.intersection(set_kleiner_accs_100_match))}")
print(f"Number of unidentified ACCs with match = 100% with blast und Hyperscore >= 23.7: {len(set_kleiner_accs_of_unidentified_spectra_23_th.intersection(set_kleiner_accs_100_match))}")

set_spectra_in_uni_species_below_23_7 = set(df_uni_spec_ref_th[df_uni_spec_ref_th.Hyperscore < 23.7].Title)
evalues = {float(item[1:-1])  for item in ref_df_psms_that_should_be_contained_in_t2P.EValue}
print("Median des Evalues für nicht identifizierte PSMs: ", median(evalues))

Number SPECTRA not identified with uniprot species DB: 4830
davon number SPECTRA unter Hyperscore von 23.7: 4130
davon number SPECTRA mit Hyperscore >= 23.7: 700
Number ACCs associate to by uniprot_species DB unidentified spectra: 4777
Number of unidentified ACCs with match < 100% with blast: 605
Number of unidentified ACCs with match < 100% with blast und Hyperscore >= 23.7: 219
Number of unidentified ACCs with match = 100% with blast: 4155
Number of unidentified ACCs with match = 100% with blast und Hyperscore >= 23.7: 401
     Hyperscore               Protein
775        77.4        {AK199_peg.96}
695        78.4  {SMS_WP_025875734.1}
615        79.9  {SMS_WP_025878597.1}
553        80.6       {AK199_peg.504}
483        81.8  {SMS_WP_025877377.1}
False
False


In [146]:
# Eigenschaften der nicht identifizierten PSMs die in unipot drin sein sollten
ref_df_psms_that_should_be_contained_in_t2P = ref_df_spectra_not_in_uniprot_spec[get_acc_rows(ref_df_spectra_not_in_uniprot_spec.Protein, proteins_that_should_be_contained_in_t2P, True)]
print(ref_df_psms_that_should_be_contained_in_t2P)

       Unnamed: 0                                          #SpecFile  \
44568       44568  /home/jules/Documents/Tax2Proteome/benchmarkin...   
44515       44515  /home/jules/Documents/Tax2Proteome/benchmarkin...   
44511       44511  /home/jules/Documents/Tax2Proteome/benchmarkin...   
44510       44510  /home/jules/Documents/Tax2Proteome/benchmarkin...   
44506       44506  /home/jules/Documents/Tax2Proteome/benchmarkin...   
...           ...                                                ...   
23624       23624  /home/jules/Documents/Tax2Proteome/benchmarkin...   
23030       23030  /home/jules/Documents/Tax2Proteome/benchmarkin...   
10662       10662  /home/jules/Documents/Tax2Proteome/benchmarkin...   
8603         8603  /home/jules/Documents/Tax2Proteome/benchmarkin...   
8335         8335  /home/jules/Documents/Tax2Proteome/benchmarkin...   

                                Title                       Peptide  \
44568    Run1_U1_2000ng.56737.56737.2                       LALD

In [159]:
# SMS_WP_025875434.1, peptide VAQLEAAVAELQAQLAGLRG ohne match in uni_spec_db, auch wenn diamon blast 100% ergeben hat
uni_spec_db[0].id
seq_match = [str(record.seq) for record in ref_db if record.id == "SMS_WP_025875434.1"][0]
id_match = [str(record.id) for record in uni_spec_db if 'PGEAPTTATSASPR' in str(record.seq)]
id_match                                                      

['tr|Q1LRQ8|Q1LRQ8_CUPMC']

In [211]:
# dauert 3h
from Bio.Seq import Seq
def search_db_for_peptide(db, peptide_set):
    len_db= len(db)
    peptide_list =list(peptide_set)
    # peptide_set = {Seq(pep) for pep in peptide_set}
    print(len_db)
    new_dict= defaultdict(set)
    for i, record in enumerate(db):
        if i%200000 == 0:
            print(i)
        l = [pep in record.seq for pep in peptide_list]
        if any(l):
            indices = [i for i, x in enumerate(l) if x]
            for i in indices:
                new_dict[peptide_list[i]].add(record.id) 
           # print(new_dict)
    return new_dict
pep_to_seq_ids_dict = search_db_for_peptide(uni_spec_db, set(ref_df_psms_that_should_be_contained_in_t2P.Peptide))
#print("Number PEPTIDES die in uniprot species enthalten sollten nach Blast: ", len(set(ref_df_psms_that_should_be_contained_in_t2P.Peptide)))
#print("Number PEPTIDES die in uniprot species DB enthalten sind: ", len(pep_to_acc_set_dict))
#list(pep_to_acc_set_dict.values())[0:2]


2991843
0
200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
2800000


In [223]:
print(len(set(ref_df_psms_that_should_be_contained_in_t2P.Peptide)))
print(len(pep_to_seq_ids_dict))
print('VAQLEAAVAELQAQLAGLRG' in pep_to_seq_ids_dict.keys()) # False
print('PGEAPTTATSASPR' in pep_to_seq_ids_dict.keys())
miss_pep=list(set(ref_df_psms_that_should_be_contained_in_t2P.Peptide).difference(set(pep_to_seq_ids_dict.keys())))
print(miss_pep)
# nur 7 waren nicht in DB drin, die laut diamond drin sein sollten
ref_df_psms_that_should_be_contained_in_t2P[(ref_df_psms_that_should_be_contained_in_t2P.Hyperscore>=23.7) & (~ref_df_psms_that_should_be_contained_in_t2P.isin(miss_pep))]
# 290 fehlen über 23.7 obwohl in DB enthalten

3378
3371
False
True
['MTTWPTVTLWAMTTASLHITLITETFPPEINGVANTLGR', 'DPAGPVLHAAGPVRPQIDGAGVMPR', 'LKEADNERQEVDNGQQQK', 'GLIEAAKAQYDRFPGYHAFFGACPTRP', 'VAQLEAAVAELQAQLAGLRG', 'NQLGNLDTTAPDAPK', 'YPISEGEDGIGTLNK']


SystemError: <built-in method view of numpy.ndarray object at 0x7f7c566f4e10> returned a result with an error set

In [ ]:
#number sequences bacillus subtilis
def search_db_for_taxon(db, taxon_graph, taxid):
    len_db= len(db)
    taxid_list =list(taxon_graph.find_(taxid))
    print(len_db)
    count = 0
    for i, record in enumerate(db):
        if i%200000 == 0:
            print(i)
        l = [f"OX={taxon}" in record.desc for taxon in taxid_list]
        if any(l):
            indices = [i for i, x in enumerate(l) if x]
            count+=1
           # print(new_dict)
    return count

In [187]:
def check_ref_peptides_for_trypsin_cut(set_peptides, db):
    new_dict= {}
    no_match =[]
    for pep in set_peptides:
        # check last pos R oder K (Arginin oder Lysin)
        if pep[-1] != 'R' and pep[-1] != 'K':
            new_dict[pep] = [True, False]
        else:
            new_dict[pep] = [True, True]
        # check pos before peptide
        seq_match = [str(record.seq) for record in db if pep in str(record.seq)]
        try:
            positions_before_match = [seq.partition(pep)[0][-1] for seq in seq_match]        
            if not 'K' in positions_before_match and not 'R'in positions_before_match:
                new_dict[pep][0] = False
        except IndexError:
            no_match.append(pep)
    return new_dict, no_match
pep_to_trypsin_start_end_dict, no_match = check_ref_peptides_for_trypsin_cut(set(ref_df_psms_that_should_be_contained_in_t2P.Peptide), ref_db)
# pep_to_trypsin_start_end_dict

{'IDIISAAMLK': [True, True],
 'QSVDPLANKPRAAGPSAGTVFPIYTPEEVEAILNANK': [True, True],
 'SSLQFIDPK': [True, True],
 'IREGWPIGVK': [True, True],
 'EDGLVYMVLEYGDIDLARLLHSHEEAR': [True, True],
 'GVAVSSNIIK': [True, True],
 'EVFGDNSPAISATK': [True, True],
 'LNVLDLQR': [True, True],
 'LELVIQR': [True, True],
 'MANQANIPVITLDR': [True, True],
 'ILEIEGLPDLK': [True, True],
 'IPIIIGR': [True, True],
 'FLVQDGVADR': [True, True],
 'FLPANEAMSDAVGLGLR': [True, True],
 'GNGSTEDLTAR': [True, True],
 'LDQIEGR': [True, True],
 'LDPEKSALGNMPLLFDPR': [True, True],
 'IRLSSAVVVLSSDDPIRVFQQFATLDLLSGGR': [True, True],
 'ASGVVRSDDGALSVDLR': [True, True],
 'ALLLTGSGR': [True, True],
 'ATVNLVPGDEIAADDEGESESLVLR': [True, True],
 'EESDLEKTEPATAR': [False, True],
 'AYVEHFIQGDPDLAK': [True, True],
 'KEDLLVPR': [True, True],
 'SPYGLNVQR': [True, True],
 'TGIFPAVFGR': [True, True],
 'VDTGAEKRAAEALNK': [True, True],
 'QSLSAVIK': [True, True],
 'AELYTDAQKR': [True, True],
 'MVYAATQEHGK': [True, True],
 'AAALANKELQTIPK': 

In [189]:
wrong_peptides = {key:value for (key,value) in pep_to_trypsin_start_end_dict.items() if value != [True, True]}
correct_peptides = {key:value for (key,value) in pep_to_trypsin_start_end_dict.items() if value == [True, True]}
print(len(wrong_peptides))
print(len(correct_peptides))
print(len(no_match))
wrong_pep_uni_spec = [pep for pep in set(df_in_fdr_uniprot_species.Peptide) if pep[-1] not in ['R','K']]
print(len(wrong_pep_uni_spec))
#wrong_pep_uni_spec

266
3112
46
419


In [ ]:
d = search_db_for_peptide(uniprot_spec_db, {"PGEAPTTATSASPR"})
print(d)
# test enthaltene auf trypsin schnittstellen:
pep_to_trypsin_start_end_dict, no_match = check_ref_peptides_for_trypsin_cut(set(pep_to_acc_set_dict.keys()), ref_db)
# einige getestet, deren peptide sind wirklich in sequenz DB drin, wurden aber nicht identifiziert, keine Ahnung warum

In [ ]:
# anzahl der T2P unbekannten Peptide bei der Ref Identifizierung
ref_identifiziert_nicht_in_t2p_df = df_in_fdr_ref[(df_in_fdr_ref.Hyperscore > 23.7) & \
                                                  (~ df_in_fdr_ref.Title.isin(set(df_in_fdr_uniprot_species.Title))) &\
                                                 (get_acc_rows(df_in_fdr_ref.Protein, acc_set_only_in_ref, True))]
print(len(set(ref_identifiziert_nicht_in_t2p_df.Title)))
print(len(set(df_in_fdr_ref[(df_in_fdr_ref.Hyperscore < 23.7)].Title)))
print(len(set(df_in_fdr_ref[(df_in_fdr_ref.Hyperscore > 23.7) & \
                            (~ df_in_fdr_ref.Title.isin(set(df_in_fdr_uniprot_species.Title)))].Title)))
print(len(set(ref_identifiziert_nicht_in_t2p_df[ref_identifiziert_nicht_in_t2p_df.Peptide.isin(list(pep_to_acc_set_dict.keys()))].Title)))

In [ ]:
# Unterschiede ref identification vs. uniprot_species T2P identifications:
# #spectra in ref < 23.7 = 4542
# # spectra in ref identifiziert mittels t2p unbekannter seq: 392
# insgesamt # spectra nicht in t2b identifiziert: 4830, davon mit Hyperscore >23.7: 568
# bleiben 152 spectra, deren peptide auch in T2P drin waren, die haben einen mean Hyperscore von 27.1, Evalue: 0.36
# nicht identifizierte Peptide: 169, davon waren 144 in der T2P DB enthalten


In [ ]:
############ TANCA ###################

In [27]:
# loading 9MM species and family:
fdr=0.05
tanca_species_df = get_df_in_fdr_without_decoy("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv/9MM_FASP_uniprot_Tanca_species_nr.t.xml_new_reduced.tsv", fdr, columns=['taxID_species'])
tanca_family_df = get_df_in_fdr_without_decoy("/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv/9MM_FASP_uniprot_Tanca_family_nr.t.xml_new_reduced.tsv", fdr, columns=['taxID_family'])

In [72]:
list(tanca_species_df.Protein.to_list()[0])[0]

'tr|A0A075R7N1|A0A075R7N1_BRELA'

In [86]:
# rhodotorula glutinis spec: 5535, family: Sporidiobolaceae 1799696
df_family_1799696_not_in_species_df, df_exclusive_new_psms_1799696 = describe_difference_between_species_and_family_identification\
("rhodotorula glutinis", 5535, 1799696, tanca_species_df, tanca_family_df)
# rhodotorula glutinis
# identified spectra species: 15 identified spectra family: 495
# new identified spectra family: 482
# identified only for taxon: 299
# all spectra counts per taxa: {1799696: 497, 4893: 151, 186822: 62, 543: 57, 33958: 44, 712: 36, 81852: 51, 'DECOY': 12, 'CRAP': 1}
# davon wurde bereits vorher für andere spezien identifiziert: 0 spektren.
# print(df_family_72275_not_in_species_df)
get_new_spectra_characteristics(1799696, df_family_1799696_not_in_species_df, tanca_family_df, tanca_species_df)
# Anzahl Protein-Identifikationen auf Family Level für neu hinzugekommen accs 
# mit 1 identifikation: 313, mit 2 identifikationen: 166,
#    mit 3 identifikationen: 63, mit 4 identifikationen: 42, 
#    mit mehr als 4 identifikationen: 107
#In species df identified PSM with count_1: 50
#In species df identified PSM with count_2: 62
#In species df identified PSM with count_3: 46
#In species df identified PSM with count_4: 51
#In species df identified PSM with count_5: 176
print('spectra characteristica for exclusive spectra: ')
get_new_spectra_characteristics(1799696, df_exclusive_new_psms_1799696, df_in_fdr_uniprot_family, df_in_fdr_uniprot_species)
exclusive_psms_of_family_level_in_species(df_exclusive_new_psms_1799696, df_in_fdr_uniprot_species, 1799696)

set_accs = {acc for acc_set in df_exclusive_new_psms_1799696.Protein for acc in acc_set}
print('Anzahl der neu identifizerten Accs, der exklusiven Spektra ', len(set_accs))
result_dict = count_psm_accs(df_in_fdr_uniprot_family, set_accs)
print(f"Accs mit einem PSM: {result_dict[1]}, Accs mit 2 PSM: {result_dict[2]}, Accs mit 3 PSM: {result_dict[3]}, Accs mit mehr als 4 PSM: {result_dict['>4']}")



NameError: name 'tanca_species_df' is not defined